In [68]:
import time as t
import re
import numpy as np
import pandas as pd
import pickle as pk
from Rider import Rider
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [2]:
STRAVA_URL = 'https://www.strava.com'

password = r'1357WorkWork!'
p = '1357Worker!'
usernames = [
    r'kvspek@gmail.com',
    r'spektork@post.bgu.ac.il',
    r'kev_backup@yahoo.com',
    r'cool_table@yahoo.com',
    r'cool_table2@yahoo.com',
    r'cool_table3@yahoo.com',
    r'cool_table4@yahoo.com',
    r'cool_table5@yahoo.com',
]

username_backup = r'kev_backup@yahoo.com'
password = r'1357Backup!'

## Prepare chrome driver and sign in to Strava

In [3]:
def chrome_driver(username, password):

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')


    # start chrome browser
    browser = webdriver.Chrome(options = chrome_options) #.Firefox(options=options)
   
    browser.get(STRAVA_URL + '/login')
    email = browser.find_element_by_id("email")
    password = browser.find_element_by_id("password")

    email.send_keys(username)
    password.send_keys(password)
    
    browser.find_element_by_id("login-button").click()
    return browser

### Date helper functions

In [4]:
def to_hours(time_string):
    time = time_string.split(':')
    total = 0
    for i in range(len(time)):
        total += int(re.sub('\D', '', time[i]))/(60**i)
    return total

def to_seconds(time_string):
    time = time_string.split(':')
    total = 0
    for i in range(len(time)):
        total += int(re.sub('\D', '', time[i]))*(60**(len(time) - i - 1))
    return total

### Extract points from elevation chart

In [6]:
def extract_points_from_graph(soup):
    svg = soup.find_all('defs')
    y_offset = float(svg[0].find('rect')['y'])
    height = float(svg[0].find('rect')['height']) + y_offset
    width = float(svg[0].find('rect')['width'])
    
    ticks = soup.find_all('g', {'class': "tick"})
    ticks = list(map(lambda x: [x['transform'].replace("translate", "")[1:-1], x.find('text').text], ticks))
    ticks = list(map(lambda x: [x[0].split(','), x[1]], ticks))
    
    x_ticks = [x for x in ticks if 'mi' in x[1]][:7]
    x_single_tick_size = float(x_ticks[1][0][0])
    miles_single_tick_size = float(x_ticks[1][1][:-3])
    ratio_x = miles_single_tick_size / x_single_tick_size

    y_ticks = [x for x in ticks if 'ft' in x[1]]
    y_single_tick_size = float(y_ticks[0][0][1]) - float(y_ticks[1][0][1])
    feet_single_tick_size = float(y_ticks[1][1][:-3].replace(",", "")) - float(y_ticks[0][1][:-3].replace(",", ""))
    
   # print(y_ticks)
   # print(feet_single_tick_size, y_single_tick_size)
    
    minimum_feet_tick = float(y_ticks[0][1][:-3].replace(",", ""))
    
    ratio_y = feet_single_tick_size / y_single_tick_size

    
    all_points = soup.find('path', {'id' : 'line'})['d'].split(',')[1:-1]
    all_points = [x.split('L') for x in all_points]
    all_points = [[height - float(x[0]), float(x[1])] for x in all_points]
    all_points = [[(x[0]*ratio_y) + minimum_feet_tick, x[1]*ratio_x] for x in all_points]
    
#     print(f'max height: {ratio_y*height + minimum_feet_tick}, max distance: {ratio_x*width}')
    return all_points

### Extract points from graph charts and calculate the mean max metrics

In [63]:
def extract_mean_max_metrics(soup):
    
    # hr_5_seconds, hr_10_seconds, hr_12_seconds, hr_20_seconds, hr_30_seconds, hr_1_minute, hr_5_minutes, hr_6_minutes, hr_10_minutes
    #all_points = extract_points_from_graph(soup)
    axis = soup.find('g', {'class':'axis xaxis'})
    ticks = axis.find_all('g', {'class': 'tick'})
    seconds_translation = []
    for tick in ticks:
        time = list(reversed(tick.find('text').text.replace('s', '').split(':'))) 
        translation = float(tick['transform'].replace('translate', '').replace('(', '').replace(')', '').split(',')[0])
        seconds = 0
        # calculate seconds
        for i in range(0, len(time)):
            seconds += int(time[i]) * 60**i
        seconds_translation.append((seconds, translation))
    single_sec_interval = (seconds_translation[1][0] - seconds_translation[0][0]), (seconds_translation[1][1] - seconds_translation[0][1])
    
    total_seconds = 0
    width_figure = 696
    height_figure = 70
    one_sec_width = single_sec_interval[1] / single_sec_interval[0]
    total_seconds = width_figure / one_sec_width
    
    def mean_metrics(points, seconds):
        current_interval = []
        max_ = []
        for point in points:
            if len(current_interval) == seconds:
                max_.append(np.mean(current_interval))
                current_interval = []
            else:
                current_interval.append(point[0])  
        return np.max(max_)
    
    def create_points(box_name, graph_name, normalize = 1):
        chart = soup.find('div', {'class': 'base-chart'})
        chart_height = float(chart.svg['height'])
        boxes = soup.find_all('g', {'class': 'label-box'})
        translations = []
        
        for box in boxes:

            translation = float(box['transform'].replace('translate', '').replace('(', '').replace(')', '').replace(' ', '').split(',')[-1])
            translations.append(translation)
           
        point_box = []
        i = -1
        for box in boxes:
            label = box.find('text', {'class': 'static-label-box label'}).text
            i += 1
            if not label == box_name:
                continue
            max_v = float(box.find('text', {'class': 'static-label-box top'}).text.replace(',', ''))
            low_v = float(box.find('text', {'class': 'static-label-box bottom'}).text.replace(',', ''))
            begin_translation = boxes[i]
            
            if i+1 < len(boxes):
                end_translation = translations[i+1] - 1
            else:
                end_translation = translations[i] + height_figure - 1
            
            point_box.append(begin_translation)
            point_box.append(end_translation)
            point_box.append(low_v)
            point_box.append(max_v)
           
            break
            
            
            
        points = soup.find('g', {'id':graph_name}).find('path')['d'].split(',')[2:-1]
        points = [x.split('L') for x in points]
        points = [[float(x[0]), float(x[1])] for x in points]
        begin_translation = point_box[0]
        end_translation = point_box[1]
        lowest_value = point_box[2]
        maximum_value = point_box[3]
        value_range = maximum_value - lowest_value
        original_points = []
                
        for point in points:
            calculated_point_y = ((end_translation - point[0])/height_figure * value_range) + lowest_value
            calculated_point_x = round((point[1]/width_figure)*total_seconds)
            original_points.append((calculated_point_y, calculated_point_x))
        points = []
        values = []
        for i in range(0, len(original_points)-1):

            point1 = original_points[i]
            point2 = original_points[i+1]

            points.append(point1)

            seconds = point2[1] - point1[1]
            point_interval = point2[0] - point1[0]
            increment = point_interval / seconds
            for j in range(1, seconds):
                new_point = ( point1[0] + increment*j ) * normalize
                points.append((new_point, point1[1]+j))
                values.append(new_point)
        
        return points, np.mean(values), np.max(values)

    dic = {}
    titles = {'5_seconds': 5, 
              '10_seconds': 10, 
              '12_seconds': 12, 
              '20_seconds': 20, 
              '30_seconds': 30, 
              '1_minute': 60, 
              '2_minutes': 120, 
              '5_minutes': 300, 
              '6_minutes': 360, 
              '10_minutes': 600,
              '12_minutes': 720,
              '20_minutes': 1200,
              '30_minutes': 1800,
              '1_hour': 3600,
              '90_minutes': 5400,
              '3_hours': 10800
             }
    
    try:
        temp, mean_temp, max_temp = create_points('Temperature' ,'temp')
    except:
        temp = [(0, 0)]
    
    dic['temp_min'] = min(temp, key = lambda x: x[0])[0]
    dic['temp_max'] = max(temp, key = lambda x: x[0])[0]
    
    try:
        speed, mean_speed, max_speed = create_points('Speed' ,'velocity_smooth')
    except:
        speed, mean_speed, max_speed = [], None, None
    
    dic[f'speed_average'] = mean_speed    
    dic[f'speed_maximum'] = max_speed
    
    try:
        watts, mean_watts, max_watts = create_points('Est Power', 'watts_calc')
    except:
        try:
            watts, mean_watts, max_watts = create_points('Power', 'watts')
        except:
            watts, mean_watts, max_watts = [], None, None
    dic[f'power_average'] = mean_watts
    dic[f'power_maximum'] = max_watts
        
    try:
        heart_rate, mean_heart_rate, max_heart_rate = create_points('Heart Rate','heartrate')
    except:
        heart_rate, mean_heart_rate, max_heart_rate = [], None, None
    dic[f'hr_average'] = mean_heart_rate
    dic[f'hr_maximum'] = max_heart_rate
    
    try:
        cadence, mean_cadence, max_cadence = create_points('Cadence', 'cadence')
    except:
        cadence, mean_cadence, max_cadence = [], None, None
    dic[f'cadence_average'] = mean_cadence
    dic[f'cadence_maximum'] = max_cadence
       
    for key, value in titles.items():
        try: 
            metric = mean_metrics(speed, value)
        except:
            metric = None
        dic[f'speed_{key}'] = metric
 

    for key, value in titles.items():
        try:
            metric = mean_metrics(watts, value)
        except:
            metric = None
        dic[f'power_{key}'] = metric
       
      
    for key, value in titles.items():
        try:
            metric = mean_metrics(heart_rate, value)
        except:
            metric = None
        dic[f'hr_{key}'] = metric
        
       
    for key, value in titles.items():
        try:
            metric = mean_metrics(cadence, value)
        except:
            metric = None
        dic[f'cadence_{key}'] = metric
       
       
    return dic


In [64]:
html = open('data/static html strava.txt')
html3 = '<div id="stacked-chart"><div class="base-chart"><svg width="966" height="381"><g id="top-container-423" transform="translate(150, 30)" class="topGroup"></g><g class="chartGroup" id="main-container-945" transform="translate(150, 30)"><g id="grid"><rect x="0" y="0" height="420" width="696" class="event-rect" style="fill: transparent;"></rect></g><g id="velocity_smooth" clip-path="url(#clip534)"><path fill="none" stroke-width="1.5px" class="simple-line" d="M0,70L1.0790697674418603,70L2.1581395348837207,32.106299227072L3.2372093023255815,30.540443823232003L4.316279069767441,29.28775950016L5.395348837209302,28.34824625785599L6.474418604651163,27.72190409632001L7.553488372093023,28.035075177087997L8.632558139534883,28.34824625785599L9.711627906976744,28.34824625785599L10.790697674418604,28.34824625785599L11.869767441860464,28.974588419392L12.948837209302326,33.672154630912L14.027906976744186,30.853614903999997L15.106976744186046,31.47995706553599L16.186046511627907,31.793128146304L17.265116279069765,32.106299227072L18.344186046511627,32.106299227072L19.42325581395349,32.419470307839994L20.502325581395347,32.732641388608L21.58139534883721,32.732641388608L22.66046511627907,32.419470307839994L23.73953488372093,31.793128146304L24.81860465116279,31.793128146304L25.89767441860465,31.166785984768005L26.97674418604651,30.540443823232003L28.05581395348837,30.227272742464002L29.134883720930233,30.227272742464002L30.21395348837209,29.914101661696L31.293023255813953,30.540443823232003L32.372093023255815,31.166785984768005L33.45116279069767,31.166785984768005L34.53023255813953,31.47995706553599L35.609302325581396,31.47995706553599L36.688372093023254,31.47995706553599L37.76744186046511,31.47995706553599L38.84651162790698,31.47995706553599L39.925581395348836,31.47995706553599L41.004651162790694,31.47995706553599L42.08372093023256,31.47995706553599L43.16279069767442,31.47995706553599L44.241860465116275,39.30923408473599L45.32093023255814,23.963851127104007L46.4,31.793128146304L47.47906976744186,31.47995706553599L48.55813953488372,31.793128146304L49.63720930232558,32.419470307839994L50.71627906976744,33.045812469375996L51.7953488372093,33.985325711680005L52.87441860465116,35.238010034752L53.95348837209302,36.177523277056L55.032558139534885,36.803865438592L56.11162790697674,37.743378680895994L57.1906976744186,38.369720842432L58.269767441860466,38.682891923199996L59.348837209302324,38.369720842432L60.42790697674418,38.682891923199996L61.50697674418605,38.682891923199996L62.586046511627906,38.682891923199996L63.665116279069764,38.682891923199996L64.74418604651163,38.369720842432L65.8232558139535,38.682891923199996L66.90232558139535,39.30923408473599L67.98139534883721,39.93557624627199L69.06046511627906,41.18826056934401L70.13953488372093,42.44094489241599L71.21860465116279,42.127773811648L72.29767441860464,42.127773811648L73.37674418604651,41.81460273088L74.45581395348837,41.18826056934401L75.53488372093022,40.561918407808L76.61395348837209,39.93557624627199L77.69302325581396,39.30923408473599L78.7720930232558,38.682891923199996L79.85116279069767,38.369720842432L80.93023255813954,38.056549761664L82.00930232558139,38.056549761664L83.08837209302325,37.743378680895994L84.16744186046512,37.743378680895994L85.24651162790697,37.43020760012799L86.32558139534883,37.43020760012799L87.4046511627907,36.803865438592L88.48372093023255,36.490694357824005L89.5627906976744,36.177523277056L90.64186046511628,36.177523277056L91.72093023255815,36.177523277056L92.8,37.11703651936L93.87906976744185,37.43020760012799L94.95813953488371,37.743378680895994L96.03720930232559,38.056549761664L97.11627906976744,38.056549761664L98.1953488372093,38.369720842432L99.27441860465116,38.682891923199996L100.35348837209303,37.743378680895994L101.43255813953488,36.490694357824005L102.51162790697673,36.177523277056L103.5906976744186,35.55118111552L104.66976744186047,34.924838953984L105.74883720930232,34.61166787321599L106.82790697674417,34.61166787321599L107.90697674418604,34.61166787321599L108.98604651162792,34.924838953984L110.06511627906977,35.55118111552L111.14418604651162,35.864352196288L112.22325581395349,36.490694357824005L113.30232558139535,37.11703651936L114.3813953488372,37.43020760012799L115.46046511627905,44.006800296255996L116.53953488372093,34.924838953984L117.6186046511628,38.056549761664L118.69767441860465,38.056549761664L119.7767441860465,38.369720842432L120.85581395348837,38.369720842432L121.93488372093024,38.369720842432L123.0139534883721,37.743378680895994L124.09302325581395,38.056549761664L125.17209302325581,37.43020760012799L126.25116279069768,37.11703651936L127.33023255813953,36.803865438592L128.4093023255814,36.803865438592L129.48837209302326,38.682891923199996L130.56744186046512,40.875089488575995L131.646511627907,42.754115973184L132.72558139534883,43.38045813472L133.8046511627907,43.38045813472L134.88372093023256,43.067287053952L135.96279069767442,42.127773811648L137.04186046511626,41.81460273088L138.12093023255812,41.18826056934401L139.20000000000002,40.561918407808L140.27906976744185,40.24874732703999L141.35813953488372,39.93557624627199L142.43720930232558,40.24874732703999L143.51627906976745,40.561918407808L144.5953488372093,41.18826056934401L145.67441860465115,41.81460273088L146.75348837209302,42.754115973184L147.83255813953488,43.38045813472L148.91162790697675,44.319971377024004L149.99069767441858,44.633142457792005L151.06976744186045,44.633142457792005L152.14883720930234,44.633142457792005L153.22790697674418,44.94631353855999L154.30697674418604,44.633142457792005L155.3860465116279,43.693629215488L156.46511627906978,43.067287053952L157.5441860465116,41.81460273088L158.62325581395348,40.875089488575995L159.70232558139534,40.561918407808L160.7813953488372,40.875089488575995L161.86046511627907,41.18826056934401L162.9395348837209,40.875089488575995L164.01860465116278,41.18826056934401L165.09767441860467,41.81460273088L166.1767441860465,41.81460273088L167.25581395348837,42.754115973184L168.33488372093024,43.067287053952L169.4139534883721,43.067287053952L170.49302325581394,43.067287053952L171.5720930232558,45.885826780864L172.65116279069767,44.006800296255996L173.73023255813953,44.94631353855999L174.8093023255814,44.94631353855999L175.88837209302324,45.25948461932799L176.9674418604651,45.572655700096L178.046511627907,45.572655700096L179.1255813953488,45.25948461932799L180.2046511627907,44.94631353855999L181.28372093023256,43.693629215488L182.36279069767443,42.127773811648L183.4418604651163,40.875089488575995L184.52093023255813,39.93557624627199L185.6,40.24874732703999L186.67906976744186,41.81460273088L187.7581395348837,44.006800296255996L188.83720930232556,44.94631353855999L189.91627906976743,44.94631353855999L190.99534883720932,44.633142457792005L192.07441860465119,44.006800296255996L193.15348837209302,43.067287053952L194.2325581395349,42.754115973184L195.31162790697675,43.693629215488L196.3906976744186,44.319971377024004L197.46976744186045,43.067287053952L198.54883720930232,42.127773811648L199.62790697674419,41.501431650111996L200.70697674418605,40.561918407808L201.7860465116279,40.24874732703999L202.86511627906975,39.93557624627199L203.94418604651165,33.672154630912L205.02325581395345,38.369720842432L206.10232558139535,38.056549761664L207.1813953488372,38.056549761664L208.26046511627908,38.369720842432L209.33953488372094,38.056549761664L210.41860465116278,37.11703651936L211.49767441860465,36.490694357824005L212.5767441860465,36.177523277056L213.65581395348835,37.43020760012799L214.7348837209302,38.682891923199996L215.81395348837208,40.24874732703999L216.89302325581397,41.18826056934401L217.97209302325584,41.18826056934401L219.05116279069767,40.875089488575995L220.13023255813954,40.24874732703999L221.2093023255814,38.369720842432L222.28837209302324,36.177523277056L223.3674418604651,34.61166787321599L224.44651162790697,33.358983550144L225.52558139534884,33.672154630912L226.6046511627907,33.358983550144L227.68372093023254,33.358983550144L228.7627906976744,33.672154630912L229.8418604651163,33.358983550144L230.9209302325581,33.672154630912L232,33.358983550144L233.07906976744187,33.045812469375996L234.15813953488373,33.672154630912L235.2372093023256,34.61166787321599L236.31627906976743,35.238010034752L237.3953488372093,35.864352196288L238.47441860465116,36.803865438592L239.553488372093,37.43020760012799L240.63255813953486,37.743378680895994L241.71162790697673,38.056549761664L242.79069767441862,38.369720842432L243.8697674418605,38.996063003968L244.94883720930233,39.30923408473599L246.0279069767442,38.682891923199996L247.10697674418606,38.682891923199996L248.1860465116279,38.682891923199996L249.26511627906976,38.682891923199996L250.34418604651162,38.996063003968L251.4232558139535,38.996063003968L252.50232558139535,38.369720842432L253.5813953488372,38.056549761664L254.66046511627906,38.056549761664L255.73953488372095,38.682891923199996L256.8186046511628,39.30923408473599L257.89767441860465,40.875089488575995L258.9767441860465,41.81460273088L260.0558139534884,42.127773811648L261.13488372093025,42.44094489241599L262.21395348837206,42.127773811648L263.293023255814,41.18826056934401L264.37209302325584,40.561918407808L265.45116279069765,39.622405165504006L266.5302325581395,37.743378680895994L267.6093023255814,35.864352196288L268.68837209302325,34.298496792448L269.7674418604651,33.358983550144L270.846511627907,33.045812469375996L271.92558139534884,34.298496792448L273.0046511627907,36.177523277056L274.0837209302325,39.30923408473599L275.1627906976744,41.501431650111996L276.24186046511625,42.44094489241599L277.32093023255817,42.754115973184L278.40000000000003,41.501431650111996L279.47906976744184,40.561918407808L280.5581395348837,39.93557624627199L281.6372093023256,38.056549761664L282.71627906976744,36.490694357824005L283.7953488372093,34.924838953984L284.87441860465117,33.672154630912L285.95348837209303,33.358983550144L287.0325581395349,32.732641388608L288.1116279069767,32.419470307839994L289.1906976744186,31.47995706553599L290.2697674418605,30.853614903999997L291.3488372093023,29.914101661696L292.42790697674417,29.914101661696L293.50697674418603,29.914101661696L294.5860465116279,30.227272742464002L295.66511627906976,29.914101661696L296.74418604651163,29.914101661696L297.8232558139535,29.28775950016L298.90232558139536,28.974588419392L299.98139534883717,28.661417338624L301.06046511627903,28.34824625785599L302.1395348837209,28.974588419392L303.2186046511628,29.28775950016L304.2976744186047,30.540443823232003L305.3767441860465,33.358983550144L306.45581395348836,35.238010034752L307.5348837209302,36.490694357824005L308.6139534883721,36.490694357824005L309.69302325581396,35.55118111552L310.7720930232558,33.672154630912L311.8511627906977,32.732641388608L312.93023255813955,32.419470307839994L314.00930232558136,31.793128146304L315.0883720930232,32.419470307839994L316.16744186046515,33.358983550144L317.24651162790695,34.61166787321599L318.3255813953488,35.55118111552L319.4046511627907,36.177523277056L320.48372093023255,43.067287053952L321.5627906976744,29.914101661696L322.6418604651163,37.11703651936L323.72093023255815,37.43020760012799L324.8,37.43020760012799L325.8790697674418,36.803865438592L326.9581395348837,35.864352196288L328.03720930232555,34.924838953984L329.1162790697675,34.61166787321599L330.19534883720934,34.61166787321599L331.27441860465115,34.61166787321599L332.353488372093,34.61166787321599L333.4325581395349,34.924838953984L334.51162790697674,34.924838953984L335.5906976744186,41.81460273088L336.6697674418605,31.793128146304L337.74883720930234,35.55118111552L338.8279069767442,37.11703651936L339.906976744186,38.056549761664L340.9860465116279,38.996063003968L342.0651162790698,39.93557624627199L343.1441860465116,40.561918407808L344.2232558139535,41.18826056934401L345.30232558139534,40.875089488575995L346.3813953488372,41.501431650111996L347.46046511627907,41.81460273088L348.53953488372093,41.81460273088L349.6186046511628,41.81460273088L350.69767441860466,41.81460273088L351.77674418604647,41.81460273088L352.8558139534884,41.81460273088L353.9348837209302,41.81460273088L355.01395348837207,41.81460273088L356.093023255814,41.81460273088L357.1720930232558,42.127773811648L358.2511627906976,42.127773811648L359.3302325581395,41.501431650111996L360.4093023255814,41.501431650111996L361.48837209302326,41.18826056934401L362.5674418604651,41.18826056934401L363.64651162790693,41.501431650111996L364.72558139534885,41.501431650111996L365.80465116279066,40.561918407808L366.8837209302326,38.682891923199996L367.96279069767445,37.11703651936L369.04186046511626,36.490694357824005L370.1209302325582,34.924838953984L371.2,33.672154630912L372.27906976744185,33.045812469375996L373.3581395348837,33.045812469375996L374.4372093023256,34.924838953984L375.5162790697674,34.298496792448L376.5953488372093,34.61166787321599L377.6744186046511,35.238010034752L378.75348837209305,35.55118111552L379.83255813953485,36.490694357824005L380.9116279069767,36.803865438592L381.99069767441864,37.743378680895994L383.06976744186045,36.803865438592L384.14883720930237,35.238010034752L385.2279069767442,34.61166787321599L386.30697674418604,38.056549761664L387.3860465116279,33.045812469375996L388.4651162790698,31.47995706553599L389.5441860465116,30.540443823232003L390.6232558139535,31.166785984768005L391.7023255813953,32.732641388608L392.7813953488372,34.298496792448L393.8604651162791,34.61166787321599L394.9395348837209,34.61166787321599L396.01860465116283,34.61166787321599L397.09767441860464,36.177523277056L398.17674418604645,33.358983550144L399.25581395348837,42.754115973184L400.33488372093024,44.633142457792005L401.4139534883721,46.198997861632L402.49302325581397,52.149248396223996L403.5720930232558,54.967788123135996L404.6511627906977,41.18826056934401L405.7302325581395,32.732641388608L406.80930232558137,26.782390854015993L407.8883720930233,22.711166804031997L408.9674418604651,22.711166804031997L410.0465116279069,23.650680046336L411.12558139534883,24.277022207872L412.2046511627907,23.963851127104007L413.28372093023256,24.277022207872L414.3627906976744,26.15604869248L415.44186046511624,27.095561934784L416.52093023255816,27.72190409632001L417.59999999999997,27.72190409632001L418.6790697674419,27.72190409632001L419.75813953488375,27.095561934784L420.83720930232556,25.216535450175996L421.9162790697675,23.963851127104007L422.9953488372093,23.337508965568L424.0744186046511,23.337508965568L425.153488372093,24.277022207872L426.2325581395349,23.337508965568L427.3116279069767,23.650680046336L428.3906976744186,25.216535450175996L429.4697674418604,27.095561934784L430.54883720930235,28.34824625785599L431.62790697674416,28.974588419392L432.706976744186,28.34824625785599L433.78604651162794,25.842877611712005L434.86511627906975,23.337508965568L435.9441860465117,22.084824642495988L437.0232558139535,21.145311400192L438.10232558139535,19.266284915584002L439.1813953488372,18.639942754048008L440.2604651162791,17.700429511743998L441.3395348837209,17.074087350207996L442.4186046511628,16.134574107904L443.4976744186046,15.195060865600006L444.5767441860465,15.195060865600006L445.65581395348835,14.881889784831984L446.7348837209302,14.881889784831984L447.81395348837214,15.508231946368L448.89302325581394,16.760916269440003L449.97209302325575,18.01360059251199L451.0511627906977,19.266284915584002L452.13023255813954,20.51896923865599L453.2093023255814,22.084824642495988L454.28837209302327,23.650680046336L455.3674418604651,24.590193288639995L456.446511627907,25.216535450175996L457.5255813953488,24.590193288639995L458.6046511627907,23.963851127104007L459.6837209302326,23.024337884799998L460.7627906976744,22.397995723264003L461.8418604651162,22.084824642495988L462.92093023255813,21.458482480960008L464,21.458482480960008L465.07906976744187,21.771653561728L466.15813953488373,22.711166804031997L467.23720930232554,24.277022207872L468.31627906976746,25.529706530943997L469.39534883720927,26.15604869248L470.4744186046512,25.842877611712005L471.553488372093,25.216535450175996L472.63255813953486,24.277022207872L473.7116279069768,23.650680046336L474.7906976744186,23.963851127104007L475.8697674418604,24.590193288639995L476.9488372093023,25.842877611712005L478.0279069767442,27.095561934784L479.106976744186,28.34824625785599L480.1860465116279,28.974588419392L481.26511627906973,29.600930580927994L482.34418604651165,29.914101661696L483.42325581395346,30.540443823232003L484.5023255813953,30.227272742464002L485.58139534883725,30.540443823232003L486.66046511627906,31.166785984768005L487.739534883721,30.853614903999997L488.8186046511628,30.540443823232003L489.89767441860465,30.227272742464002L490.9767441860465,30.227272742464002L492.0558139534884,30.227272742464002L493.1348837209302,29.914101661696L494.2139534883721,37.743378680895994L495.2930232558139,25.216535450175996L496.3720930232558,27.72190409632001L497.45116279069765,25.529706530943997L498.5302325581395,23.024337884799998L499.60930232558144,20.51896923865599L500.68837209302325,19.89262707711999L501.76744186046506,21.145311400192L502.846511627907,22.711166804031997L503.92558139534884,25.216535450175996L505.0046511627907,26.469219773247993L506.0837209302326,26.469219773247993L507.1627906976744,25.842877611712005L508.2418604651163,23.963851127104007L509.3209302325581,22.084824642495988L510.4,20.205798157887997L511.4790697674419,18.953113834815994L512.5581395348837,18.639942754048008L513.6372093023256,18.953113834815994L514.7162790697674,29.28775950016L515.7953488372093,14.881889784831984L516.8744186046512,20.832140319424006L517.953488372093,21.771653561728L519.0325581395349,22.084824642495988L520.1116279069768,22.397995723264003L521.1906976744186,22.084824642495988L522.2697674418605,22.084824642495988L523.3488372093024,22.084824642495988L524.4279069767441,21.771653561728L525.5069767441861,20.832140319424006L526.586046511628,20.205798157887997L527.6651162790697,20.51896923865599L528.7441860465117,20.832140319424006L529.8232558139534,21.145311400192L530.9023255813953,21.458482480960008L531.9813953488372,21.771653561728L533.060465116279,22.397995723264003L534.1395348837209,23.024337884799998L535.2186046511628,23.024337884799998L536.2976744186046,23.337508965568L537.3767441860465,22.711166804031997L538.4558139534884,20.832140319424006L539.5348837209302,18.639942754048008L540.6139534883721,17.700429511743998L541.693023255814,18.32677167328L542.7720930232558,19.579455996351996L543.8511627906977,19.266284915584002L544.9302325581396,19.266284915584002L546.0093023255814,19.579455996351996L547.0883720930233,20.205798157887997L548.167441860465,21.458482480960008L549.246511627907,21.771653561728L550.3255813953488,22.397995723264003L551.4046511627907,22.397995723264003L552.4837209302325,21.771653561728L553.5627906976744,22.084824642495988L554.6418604651163,21.771653561728L555.7209302325581,30.540443823232003L556.8000000000001,8.618468169471988L557.8790697674418,19.579455996351996L558.9581395348837,20.832140319424006L560.0372093023256,21.458482480960008L561.1162790697674,21.771653561728L562.1953488372093,21.145311400192L563.2744186046511,20.205798157887997L564.353488372093,18.953113834815994L565.4325581395349,18.639942754048008L566.5116279069767,18.01360059251199L567.5906976744186,17.074087350207996L568.6697674418605,17.387258430976004L569.7488372093023,18.32677167328L570.8279069767442,19.266284915584002L571.9069767441861,20.205798157887997L572.9860465116279,21.145311400192L574.0651162790698,20.832140319424006L575.1441860465117,19.579455996351996L576.2232558139534,18.32677167328L577.3023255813954,17.074087350207996L578.3813953488371,15.821403027135986L579.460465116279,15.821403027135986L580.539534883721,17.074087350207996L581.6186046511627,18.32677167328L582.6976744186046,19.266284915584002L583.7767441860465,19.579455996351996L584.8558139534883,18.953113834815994L585.9348837209302,17.700429511743998L587.0139534883721,16.134574107904L588.0930232558139,17.387258430976004L589.1720930232558,18.639942754048008L590.2511627906977,19.266284915584002L591.3302325581395,19.579455996351996L592.4093023255814,19.89262707711999L593.4883720930233,19.579455996351996L594.5674418604651,19.579455996351996L595.646511627907,19.266284915584002L596.7255813953489,18.953113834815994L597.8046511627907,18.639942754048008L598.8837209302326,18.32677167328L599.9627906976743,18.01360059251199L601.0418604651163,18.953113834815994L602.1209302325581,19.89262707711999L603.2,20.51896923865599L604.2790697674418,21.458482480960008L605.3581395348837,22.397995723264003L606.4372093023256,22.711166804031997L607.5162790697674,22.084824642495988L608.5953488372094,21.771653561728L609.6744186046511,22.084824642495988L610.753488372093,22.397995723264003L611.8325581395349,22.397995723264003L612.9116279069767,22.397995723264003L613.9906976744186,23.024337884799998L615.0697674418604,23.024337884799998L616.1488372093023,23.337508965568L617.2279069767442,23.024337884799998L618.306976744186,23.024337884799998L619.3860465116279,22.397995723264003L620.4651162790698,22.397995723264003L621.5441860465116,22.084824642495988L622.6232558139535,21.458482480960008L623.7023255813954,21.458482480960008L624.7813953488372,21.458482480960008L625.8604651162791,20.51896923865599L626.939534883721,18.32677167328L628.0186046511627,17.074087350207996L629.0976744186047,18.01360059251199L630.1767441860464,19.579455996351996L631.2558139534883,20.832140319424006L632.3348837209303,21.458482480960008L633.413953488372,21.458482480960008L634.4930232558139,21.458482480960008L635.5720930232558,21.458482480960008L636.6511627906976,21.458482480960008L637.7302325581395,31.166785984768005L638.8093023255814,13.002863300224L639.8883720930232,23.963851127104007L640.9674418604651,25.216535450175996L642.046511627907,25.529706530943997L643.1255813953488,25.216535450175996L644.2046511627907,25.216535450175996L645.2837209302326,24.590193288639995L646.3627906976744,24.903364369408003L647.4418604651163,24.590193288639995L648.5209302325582,24.903364369408003L649.6,25.529706530943997L650.6790697674419,25.842877611712005L651.7581395348836,25.842877611712005L652.8372093023256,26.15604869248L653.9162790697674,25.842877611712005L654.9953488372093,25.842877611712005L656.0744186046511,25.529706530943997L657.153488372093,25.842877611712005L658.232558139535,25.842877611712005L659.3116279069767,26.15604869248L660.3906976744187,26.469219773247993L661.4697674418604,26.782390854015993L662.5488372093023,26.782390854015993L663.6279069767442,25.842877611712005L664.706976744186,25.216535450175996L665.7860465116279,24.590193288639995L666.8651162790698,23.650680046336L667.9441860465116,23.024337884799998L669.0232558139535,22.397995723264003L670.1023255813953,22.084824642495988L671.1813953488372,22.397995723264003L672.2604651162791,23.337508965568L673.339534883721,33.672154630912L674.4186046511627,17.387258430976004L675.4976744186047,27.095561934784L676.5767441860465,27.72190409632001L677.6558139534884,28.34824625785599L678.7348837209303,28.661417338624L679.813953488372,28.34824625785599L680.893023255814,28.34824625785599L681.9720930232558,28.661417338624L683.0511627906976,28.035075177087997L684.1302325581396,28.035075177087997L685.2093023255813,26.782390854015993L686.2883720930232,32.106299227072L687.3674418604651,38.682891923199996L688.446511627907,33.045812469375996L689.5255813953488,30.227272742464002L690.6046511627907,32.106299227072L691.6837209302325,39.93557624627199L692.7627906976744,49.017537588544L693.8418604651163,51.20973515391999L694.9209302325581,51.522906234687994L696,52.149248396223996"></path></g><g id="watts" clip-path="url(#clip132)"><path fill="none" stroke-width="1.5px" class="simple-line" d="M0,88.73846153846154L1.0790697674418603,98.86153846153846L2.1581395348837207,104.13846153846154L3.2372093023255815,100.6923076923077L4.316279069767441,107.36923076923077L5.395348837209302,103.70769230769231L6.474418604651163,104.78461538461539L7.553488372093023,107.8L8.632558139534883,107.15384615384616L9.711627906976744,105.53846153846153L10.790697674418604,108.23076923076923L11.869767441860464,104.24615384615385L12.948837209302326,103.27692307692308L14.027906976744186,100.47692307692307L15.106976744186046,97.13846153846154L16.186046511627907,105L17.265116279069765,109.2L18.344186046511627,106.50769230769231L19.42325581395349,107.90769230769232L20.502325581395347,103.81538461538462L21.58139534883721,108.01538461538462L22.66046511627907,105L23.73953488372093,110.81538461538462L24.81860465116279,107.6923076923077L25.89767441860465,104.46153846153847L26.97674418604651,105.86153846153846L28.05581395348837,110.06153846153846L29.134883720930233,108.76923076923077L30.21395348837209,107.04615384615384L31.293023255813953,109.73846153846154L32.372093023255815,104.78461538461539L33.45116279069767,106.61538461538461L34.53023255813953,105.86153846153846L35.609302325581396,102.95384615384614L36.688372093023254,104.13846153846154L37.76744186046511,109.41538461538462L38.84651162790698,103.92307692307693L39.925581395348836,104.46153846153847L41.004651162790694,109.84615384615384L42.08372093023256,109.73846153846154L43.16279069767442,111.03076923076924L44.241860465116275,110.6L45.32093023255814,111.67692307692307L46.4,107.36923076923077L47.47906976744186,107.04615384615384L48.55813953488372,107.47692307692307L49.63720930232558,105.32307692307693L50.71627906976744,106.4L51.7953488372093,109.95384615384616L52.87441860465116,109.2L53.95348837209302,107.47692307692307L55.032558139534885,109.09230769230768L56.11162790697674,112.21538461538461L57.1906976744186,111.03076923076924L58.269767441860466,104.13846153846154L59.348837209302324,96.49230769230769L60.42790697674418,108.87692307692308L61.50697674418605,112.32307692307693L62.586046511627906,108.76923076923077L63.665116279069764,109.84615384615384L64.74418604651163,108.12307692307692L65.8232558139535,108.76923076923077L66.90232558139535,102.84615384615384L67.98139534883721,111.35384615384615L69.06046511627906,108.55384615384615L70.13953488372093,102.41538461538461L71.21860465116279,107.04615384615384L72.29767441860464,98.86153846153846L73.37674418604651,108.76923076923077L74.45581395348837,105.1076923076923L75.53488372093022,108.12307692307692L76.61395348837209,104.35384615384615L77.69302325581396,110.49230769230769L78.7720930232558,108.44615384615385L79.85116279069767,96.49230769230769L80.93023255813954,111.56923076923077L82.00930232558139,111.78461538461539L83.08837209302325,111.13846153846154L84.16744186046512,108.44615384615385L85.24651162790697,105.32307692307693L86.32558139534883,111.13846153846154L87.4046511627907,110.27692307692308L88.48372093023255,107.15384615384616L89.5627906976744,115.76923076923077L90.64186046511628,108.12307692307692L91.72093023255815,109.63076923076923L92.8,111.46153846153847L93.87906976744185,108.55384615384615L94.95813953488371,109.3076923076923L96.03720930232559,109.84615384615384L97.11627906976744,109.09230769230768L98.1953488372093,108.66153846153847L99.27441860465116,104.13846153846154L100.35348837209303,108.33846153846153L101.43255813953488,110.27692307692308L102.51162790697673,108.55384615384615L103.5906976744186,112.32307692307693L104.66976744186047,110.92307692307692L105.74883720930232,107.47692307692307L106.82790697674417,109.84615384615384L107.90697674418604,111.35384615384615L108.98604651162792,108.55384615384615L110.06511627906977,109.84615384615384L111.14418604651162,108.01538461538462L112.22325581395349,113.61538461538461L113.30232558139535,108.44615384615385L114.3813953488372,111.56923076923077L115.46046511627905,109.3076923076923L116.53953488372093,108.12307692307692L117.6186046511628,110.16923076923077L118.69767441860465,107.15384615384616L119.7767441860465,111.13846153846154L120.85581395348837,112L121.93488372093024,108.44615384615385L123.0139534883721,113.07692307692308L124.09302325581395,110.92307692307692L125.17209302325581,108.33846153846153L126.25116279069768,111.03076923076924L127.33023255813953,102.52307692307693L128.4093023255814,110.70769230769231L129.48837209302326,102.84615384615384L130.56744186046512,105.64615384615385L131.646511627907,98L132.72558139534883,110.16923076923077L133.8046511627907,112.75384615384615L134.88372093023256,105.75384615384615L135.96279069767442,113.83076923076923L137.04186046511626,115.66153846153847L138.12093023255812,110.92307692307692L139.20000000000002,106.2923076923077L140.27906976744185,106.2923076923077L141.35813953488372,115.55384615384615L142.43720930232558,107.90769230769232L143.51627906976745,104.13846153846154L144.5953488372093,105.43076923076923L145.67441860465115,101.01538461538462L146.75348837209302,106.72307692307692L147.83255813953488,106.4L148.91162790697675,104.56923076923077L149.99069767441858,106.93846153846154L151.06976744186045,108.12307692307692L152.14883720930234,106.61538461538461L153.22790697674418,108.23076923076923L154.30697674418604,110.92307692307692L155.3860465116279,112.21538461538461L156.46511627906978,102.73846153846154L157.5441860465116,110.70769230769231L158.62325581395348,110.92307692307692L159.70232558139534,110.81538461538462L160.7813953488372,105.53846153846153L161.86046511627907,105.32307692307693L162.9395348837209,110.92307692307692L164.01860465116278,108.87692307692308L165.09767441860467,100.90769230769232L166.1767441860465,108.01538461538462L167.25581395348837,106.83076923076922L168.33488372093024,107.6923076923077L169.4139534883721,106.07692307692308L170.49302325581394,109.52307692307693L171.5720930232558,103.81538461538462L172.65116279069767,108.98461538461538L173.73023255813953,102.84615384615384L174.8093023255814,107.36923076923077L175.88837209302324,106.18461538461538L176.9674418604651,96.6L178.046511627907,107.15384615384616L179.1255813953488,105.96923076923076L180.2046511627907,110.92307692307692L181.28372093023256,106.07692307692308L182.36279069767443,114.04615384615384L183.4418604651163,108.87692307692308L184.52093023255813,111.67692307692307L185.6,105.32307692307693L186.67906976744186,106.72307692307692L187.7581395348837,102.19999999999999L188.83720930232556,101.44615384615385L189.91627906976743,104.13846153846154L190.99534883720932,108.44615384615385L192.07441860465119,111.24615384615385L193.15348837209302,105.53846153846153L194.2325581395349,109.52307692307693L195.31162790697675,104.56923076923077L196.3906976744186,113.61538461538461L197.46976744186045,112.32307692307693L198.54883720930232,102.84615384615384L199.62790697674419,112.86153846153846L200.70697674418605,110.06153846153846L201.7860465116279,114.47692307692307L202.86511627906975,113.4L203.94418604651165,102.19999999999999L205.02325581395345,109.73846153846154L206.10232558139535,109.3076923076923L207.1813953488372,111.78461538461539L208.26046511627908,112.1076923076923L209.33953488372094,108.12307692307692L210.41860465116278,115.66153846153847L211.49767441860465,108.33846153846153L212.5767441860465,113.83076923076923L213.65581395348835,105.32307692307693L214.7348837209302,107.15384615384616L215.81395348837208,106.61538461538461L216.89302325581397,107.04615384615384L217.97209302325584,107.90769230769232L219.05116279069767,111.13846153846154L220.13023255813954,108.44615384615385L221.2093023255814,112L222.28837209302324,104.24615384615385L223.3674418604651,104.35384615384615L224.44651162790697,111.03076923076924L225.52558139534884,111.24615384615385L226.6046511627907,110.16923076923077L227.68372093023254,113.18461538461538L228.7627906976744,108.87692307692308L229.8418604651163,106.07692307692308L230.9209302325581,116.3076923076923L232,109.73846153846154L233.07906976744187,115.66153846153847L234.15813953488373,111.56923076923077L235.2372093023256,110.6L236.31627906976743,111.24615384615385L237.3953488372093,112.32307692307693L238.47441860465116,110.16923076923077L239.553488372093,110.92307692307692L240.63255813953486,110.6L241.71162790697673,107.90769230769232L242.79069767441862,106.83076923076922L243.8697674418605,105.1076923076923L244.94883720930233,104.56923076923077L246.0279069767442,109.73846153846154L247.10697674418606,101.87692307692308L248.1860465116279,113.4L249.26511627906976,109.63076923076923L250.34418604651162,108.98461538461538L251.4232558139535,108.33846153846153L252.50232558139535,110.92307692307692L253.5813953488372,108.01538461538462L254.66046511627906,107.90769230769232L255.73953488372095,103.16923076923077L256.8186046511628,106.07692307692308L257.89767441860465,102.73846153846154L258.9767441860465,103.49230769230769L260.0558139534884,104.67692307692309L261.13488372093025,105.21538461538461L262.21395348837206,107.90769230769232L263.293023255814,113.93846153846154L264.37209302325584,119L265.45116279069765,113.61538461538461L266.5302325581395,118.67692307692307L267.6093023255814,113.18461538461538L268.68837209302325,116.73846153846154L269.7674418604651,115.44615384615385L270.846511627907,112.86153846153846L271.92558139534884,111.03076923076924L273.0046511627907,109.3076923076923L274.0837209302325,110.06153846153846L275.1627906976744,104.13846153846154L276.24186046511625,101.76923076923077L277.32093023255817,103.16923076923077L278.40000000000003,103.81538461538462L279.47906976744184,111.56923076923077L280.5581395348837,109.41538461538462L281.6372093023256,111.24615384615385L282.71627906976744,107.36923076923077L283.7953488372093,115.87692307692308L284.87441860465117,115.44615384615385L285.95348837209303,114.8L287.0325581395349,117.81538461538462L288.1116279069767,109.09230769230768L289.1906976744186,113.4L290.2697674418605,112.32307692307693L291.3488372093023,107.8L292.42790697674417,110.70769230769231L293.50697674418603,108.33846153846153L294.5860465116279,96.49230769230769L295.66511627906976,107.8L296.74418604651163,112.43076923076923L297.8232558139535,108.33846153846153L298.90232558139536,107.26153846153846L299.98139534883717,113.18461538461538L301.06046511627903,110.49230769230769L302.1395348837209,110.49230769230769L303.2186046511628,122.12307692307692L304.2976744186047,112.21538461538461L305.3767441860465,104.46153846153847L306.45581395348836,108.44615384615385L307.5348837209302,109.73846153846154L308.6139534883721,108.76923076923077L309.69302325581396,108.44615384615385L310.7720930232558,108.66153846153847L311.8511627906977,108.33846153846153L312.93023255813955,107.58461538461539L314.00930232558136,109.52307692307693L315.0883720930232,108.66153846153847L316.16744186046515,112.43076923076923L317.24651162790695,106.83076923076922L318.3255813953488,112.21538461538461L319.4046511627907,107.8L320.48372093023255,110.70769230769231L321.5627906976744,109.09230769230768L322.6418604651163,113.93846153846154L323.72093023255815,110.92307692307692L324.8,115.33846153846153L325.8790697674418,108.55384615384615L326.9581395348837,111.13846153846154L328.03720930232555,113.50769230769231L329.1162790697675,112.96923076923076L330.19534883720934,111.78461538461539L331.27441860465115,109.09230769230768L332.353488372093,104.35384615384615L333.4325581395349,112L334.51162790697674,107.36923076923077L335.5906976744186,96.6L336.6697674418605,107.90769230769232L337.74883720930234,107.90769230769232L338.8279069767442,112.75384615384615L339.906976744186,104.46153846153847L340.9860465116279,109.52307692307693L342.0651162790698,108.01538461538462L343.1441860465116,112.75384615384615L344.2232558139535,107.58461538461539L345.30232558139534,108.23076923076923L346.3813953488372,106.83076923076922L347.46046511627907,107.47692307692307L348.53953488372093,103.70769230769231L349.6186046511628,108.98461538461538L350.69767441860466,112.86153846153846L351.77674418604647,107.15384615384616L352.8558139534884,106.50769230769231L353.9348837209302,105.96923076923076L355.01395348837207,109.09230769230768L356.093023255814,96.49230769230769L357.1720930232558,106.83076923076922L358.2511627906976,105.75384615384615L359.3302325581395,112.75384615384615L360.4093023255814,101.23076923076923L361.48837209302326,110.16923076923077L362.5674418604651,107.47692307692307L363.64651162790693,108.23076923076923L364.72558139534885,109.09230769230768L365.80465116279066,112.43076923076923L366.8837209302326,112.64615384615385L367.96279069767445,113.83076923076923L369.04186046511626,109.52307692307693L370.1209302325582,113.50769230769231L371.2,110.16923076923077L372.27906976744185,115.23076923076923L373.3581395348837,103.6L374.4372093023256,109.09230769230768L375.5162790697674,111.8923076923077L376.5953488372093,107.58461538461539L377.6744186046511,104.46153846153847L378.75348837209305,109.84615384615384L379.83255813953485,106.18461538461538L380.9116279069767,111.78461538461539L381.99069767441864,109.95384615384616L383.06976744186045,106.93846153846154L384.14883720930237,115.66153846153847L385.2279069767442,115.12307692307692L386.30697674418604,102.63076923076923L387.3860465116279,108.66153846153847L388.4651162790698,113.50769230769231L389.5441860465116,114.58461538461539L390.6232558139535,112L391.7023255813953,107.04615384615384L392.7813953488372,108.44615384615385L393.8604651162791,112.53846153846155L394.9395348837209,110.27692307692308L396.01860465116283,108.33846153846153L397.09767441860464,107.90769230769232L398.17674418604645,99.4L399.25581395348837,105.43076923076923L400.33488372093024,107.15384615384616L401.4139534883721,105.64615384615385L402.49302325581397,140L403.5720930232558,91.53846153846155L404.6511627906977,116.95384615384616L405.7302325581395,115.66153846153847L406.80930232558137,114.04615384615384L407.8883720930233,114.04615384615384L408.9674418604651,115.23076923076923L410.0465116279069,113.93846153846154L411.12558139534883,114.15384615384616L412.2046511627907,96.49230769230769L413.28372093023256,105.32307692307693L414.3627906976744,107.36923076923077L415.44186046511624,116.41538461538461L416.52093023255816,105.21538461538461L417.59999999999997,113.18461538461538L418.6790697674419,108.55384615384615L419.75813953488375,116.0923076923077L420.83720930232556,109.95384615384616L421.9162790697675,109.63076923076923L422.9953488372093,115.55384615384615L424.0744186046511,111.35384615384615L425.153488372093,105.75384615384615L426.2325581395349,114.15384615384616L427.3116279069767,111.03076923076924L428.3906976744186,109.73846153846154L429.4697674418604,112.43076923076923L430.54883720930235,118.46153846153845L431.62790697674416,112.96923076923076L432.706976744186,96.06153846153846L433.78604651162794,117.16923076923078L434.86511627906975,114.47692307692307L435.9441860465117,114.26153846153846L437.0232558139535,112.53846153846155L438.10232558139535,113.07692307692308L439.1813953488372,112.21538461538461L440.2604651162791,117.06153846153846L441.3395348837209,118.35384615384615L442.4186046511628,107.58461538461539L443.4976744186046,115.23076923076923L444.5767441860465,111.78461538461539L445.65581395348835,109.63076923076923L446.7348837209302,113.4L447.81395348837214,116.95384615384616L448.89302325581394,109.09230769230768L449.97209302325575,107.47692307692307L451.0511627906977,110.6L452.13023255813954,107.26153846153846L453.2093023255814,109.73846153846154L454.28837209302327,116.2L455.3674418604651,113.83076923076923L456.446511627907,110.16923076923077L457.5255813953488,108.01538461538462L458.6046511627907,108.66153846153847L459.6837209302326,109.3076923076923L460.7627906976744,114.04615384615384L461.8418604651162,118.24615384615385L462.92093023255813,108.44615384615385L464,104.67692307692309L465.07906976744187,110.70769230769231L466.15813953488373,109.41538461538462L467.23720930232554,108.12307692307692L468.31627906976746,109.2L469.39534883720927,109.73846153846154L470.4744186046512,110.6L471.553488372093,116.52307692307693L472.63255813953486,111.03076923076924L473.7116279069768,96.6L474.7906976744186,108.55384615384615L475.8697674418604,110.70769230769231L476.9488372093023,109.52307692307693L478.0279069767442,109.52307692307693L479.106976744186,106.4L480.1860465116279,105.64615384615385L481.26511627906973,112.75384615384615L482.34418604651165,112.64615384615385L483.42325581395346,111.03076923076924L484.5023255813953,102.19999999999999L485.58139534883725,112.64615384615385L486.66046511627906,114.04615384615384L487.739534883721,114.36923076923077L488.8186046511628,104.78461538461539L489.89767441860465,117.38461538461539L490.9767441860465,105.43076923076923L492.0558139534884,104.46153846153847L493.1348837209302,106.61538461538461L494.2139534883721,111.56923076923077L495.2930232558139,116.84615384615384L496.3720930232558,116.3076923076923L497.45116279069765,118.35384615384615L498.5302325581395,118.24615384615385L499.60930232558144,115.55384615384615L500.68837209302325,115.12307692307692L501.76744186046506,109.2L502.846511627907,116.84615384615384L503.92558139534884,113.18461538461538L505.0046511627907,112.1076923076923L506.0837209302326,110.27692307692308L507.1627906976744,110.16923076923077L508.2418604651163,115.66153846153847L509.3209302325581,120.4L510.4,115.66153846153847L511.4790697674419,111.35384615384615L512.5581395348837,113.93846153846154L513.6372093023256,115.12307692307692L514.7162790697674,114.26153846153846L515.7953488372093,115.98461538461538L516.8744186046512,114.26153846153846L517.953488372093,113.50769230769231L519.0325581395349,114.90769230769232L520.1116279069768,111.78461538461539L521.1906976744186,109.63076923076923L522.2697674418605,112.43076923076923L523.3488372093024,108.87692307692308L524.4279069767441,110.70769230769231L525.5069767441861,103.38461538461539L526.586046511628,109.52307692307693L527.6651162790697,116.3076923076923L528.7441860465117,111.8923076923077L529.8232558139534,112.86153846153846L530.9023255813953,109.52307692307693L531.9813953488372,112.64615384615385L533.060465116279,106.50769230769231L534.1395348837209,110.16923076923077L535.2186046511628,106.83076923076922L536.2976744186046,119.1076923076923L537.3767441860465,115.12307692307692L538.4558139534884,109.52307692307693L539.5348837209302,109.09230769230768L540.6139534883721,113.18461538461538L541.693023255814,109.41538461538462L542.7720930232558,102.95384615384614L543.8511627906977,112.64615384615385L544.9302325581396,115.12307692307692L546.0093023255814,110.70769230769231L547.0883720930233,115.76923076923077L548.167441860465,114.6923076923077L549.246511627907,108.12307692307692L550.3255813953488,114.04615384615384L551.4046511627907,113.61538461538461L552.4837209302325,113.61538461538461L553.5627906976744,107.8L554.6418604651163,115.44615384615385L555.7209302325581,117.81538461538462L556.8000000000001,111.67692307692307L557.8790697674418,108.55384615384615L558.9581395348837,114.04615384615384L560.0372093023256,110.06153846153846L561.1162790697674,119.1076923076923L562.1953488372093,117.16923076923078L563.2744186046511,110.38461538461539L564.353488372093,110.6L565.4325581395349,114.36923076923077L566.5116279069767,106.18461538461538L567.5906976744186,110.6L568.6697674418605,113.18461538461538L569.7488372093023,112.32307692307693L570.8279069767442,114.8L571.9069767441861,109.84615384615384L572.9860465116279,119.43076923076923L574.0651162790698,113.18461538461538L575.1441860465117,111.8923076923077L576.2232558139534,114.15384615384616L577.3023255813954,112.86153846153846L578.3813953488371,114.36923076923077L579.460465116279,114.15384615384616L580.539534883721,110.27692307692308L581.6186046511627,107.47692307692307L582.6976744186046,114.04615384615384L583.7767441860465,113.50769230769231L584.8558139534883,115.98461538461538L585.9348837209302,110.16923076923077L587.0139534883721,114.47692307692307L588.0930232558139,110.16923076923077L589.1720930232558,113.07692307692308L590.2511627906977,113.29230769230769L591.3302325581395,112.96923076923076L592.4093023255814,110.38461538461539L593.4883720930233,108.23076923076923L594.5674418604651,108.23076923076923L595.646511627907,107.6923076923077L596.7255813953489,108.55384615384615L597.8046511627907,109.84615384615384L598.8837209302326,108.98461538461538L599.9627906976743,111.24615384615385L601.0418604651163,110.92307692307692L602.1209302325581,110.16923076923077L603.2,114.58461538461539L604.2790697674418,108.66153846153847L605.3581395348837,109.3076923076923L606.4372093023256,113.29230769230769L607.5162790697674,115.87692307692308L608.5953488372094,115.98461538461538L609.6744186046511,111.67692307692307L610.753488372093,108.33846153846153L611.8325581395349,111.46153846153847L612.9116279069767,116.0923076923077L613.9906976744186,111.35384615384615L615.0697674418604,106.83076923076922L616.1488372093023,114.8L617.2279069767442,113.83076923076923L618.306976744186,114.47692307692307L619.3860465116279,107.26153846153846L620.4651162790698,110.49230769230769L621.5441860465116,116.0923076923077L622.6232558139535,112.53846153846155L623.7023255813954,106.2923076923077L624.7813953488372,110.38461538461539L625.8604651162791,112.96923076923076L626.939534883721,115.87692307692308L628.0186046511627,119.1076923076923L629.0976744186047,116.63076923076923L630.1767441860464,119L631.2558139534883,110.38461538461539L632.3348837209303,96.38461538461539L633.413953488372,114.04615384615384L634.4930232558139,112.53846153846155L635.5720930232558,106.72307692307692L636.6511627906976,113.83076923076923L637.7302325581395,115.76923076923077L638.8093023255814,112.86153846153846L639.8883720930232,113.50769230769231L640.9674418604651,110.06153846153846L642.046511627907,109.73846153846154L643.1255813953488,115.33846153846153L644.2046511627907,103.38461538461539L645.2837209302326,112.53846153846155L646.3627906976744,108.23076923076923L647.4418604651163,107.6923076923077L648.5209302325582,112.53846153846155L649.6,110.6L650.6790697674419,109.95384615384616L651.7581395348836,109.63076923076923L652.8372093023256,108.44615384615385L653.9162790697674,106.72307692307692L654.9953488372093,118.56923076923077L656.0744186046511,107.6923076923077L657.153488372093,108.44615384615385L658.232558139535,107.15384615384616L659.3116279069767,102.19999999999999L660.3906976744187,111.24615384615385L661.4697674418604,107.36923076923077L662.5488372093023,107.8L663.6279069767442,108.23076923076923L664.706976744186,118.13846153846154L665.7860465116279,110.92307692307692L666.8651162790698,108.87692307692308L667.9441860465116,110.27692307692308L669.0232558139535,105.75384615384615L670.1023255813953,113.18461538461538L671.1813953488372,110.06153846153846L672.2604651162791,112.21538461538461L673.339534883721,106.61538461538461L674.4186046511627,101.44615384615385L675.4976744186047,102.63076923076923L676.5767441860465,106.83076923076922L677.6558139534884,105.53846153846153L678.7348837209303,106.61538461538461L679.813953488372,106.50769230769231L680.893023255814,116.0923076923077L681.9720930232558,106.2923076923077L683.0511627906976,107.15384615384616L684.1302325581396,112.1076923076923L685.2093023255813,114.90769230769232L686.2883720930232,140L687.3674418604651,88.41538461538462L688.446511627907,105.32307692307693L689.5255813953488,101.23076923076923L690.6046511627907,102.73846153846154L691.6837209302325,90.78461538461539L692.7627906976744,70.75384615384615L693.8418604651163,75.81538461538462L694.9209302325581,76.56923076923076L696,76.67692307692307"></path></g><g id="heartrate" clip-path="url(#clip373)"><path fill="none" stroke-width="1.5px" class="simple-line" d="M0,203L1.0790697674418603,183.55555555555554L2.1581395348837207,177.33333333333334L3.2372093023255815,175.77777777777777L4.316279069767441,172.66666666666666L5.395348837209302,171.88888888888889L6.474418604651163,171.11111111111111L7.553488372093023,169.55555555555554L8.632558139534883,169.55555555555554L9.711627906976744,170.33333333333334L10.790697674418604,169.55555555555554L11.869767441860464,169.55555555555554L12.948837209302326,169.55555555555554L14.027906976744186,168.77777777777777L15.106976744186046,168L16.186046511627907,168L17.265116279069765,168L18.344186046511627,168L19.42325581395349,168L20.502325581395347,167.22222222222223L21.58139534883721,167.22222222222223L22.66046511627907,166.44444444444446L23.73953488372093,165.66666666666669L24.81860465116279,165.66666666666669L25.89767441860465,165.66666666666669L26.97674418604651,164.88888888888889L28.05581395348837,165.66666666666669L29.134883720930233,164.88888888888889L30.21395348837209,162.55555555555554L31.293023255813953,164.88888888888889L32.372093023255815,164.88888888888889L33.45116279069767,164.88888888888889L34.53023255813953,164.88888888888889L35.609302325581396,164.88888888888889L36.688372093023254,164.11111111111111L37.76744186046511,164.11111111111111L38.84651162790698,164.11111111111111L39.925581395348836,164.11111111111111L41.004651162790694,163.33333333333331L42.08372093023256,163.33333333333331L43.16279069767442,162.55555555555554L44.241860465116275,163.33333333333331L45.32093023255814,163.33333333333331L46.4,163.33333333333331L47.47906976744186,163.33333333333331L48.55813953488372,163.33333333333331L49.63720930232558,162.55555555555554L50.71627906976744,163.33333333333331L51.7953488372093,163.33333333333331L52.87441860465116,163.33333333333331L53.95348837209302,163.33333333333331L55.032558139534885,162.55555555555554L56.11162790697674,162.55555555555554L57.1906976744186,162.55555555555554L58.269767441860466,161.77777777777777L59.348837209302324,161.77777777777777L60.42790697674418,161.77777777777777L61.50697674418605,162.55555555555554L62.586046511627906,162.55555555555554L63.665116279069764,161.77777777777777L64.74418604651163,161.77777777777777L65.8232558139535,161.77777777777777L66.90232558139535,161L67.98139534883721,161.77777777777777L69.06046511627906,161.77777777777777L70.13953488372093,162.55555555555554L71.21860465116279,161.77777777777777L72.29767441860464,161.77777777777777L73.37674418604651,161L74.45581395348837,161L75.53488372093022,161L76.61395348837209,161L77.69302325581396,161L78.7720930232558,159.44444444444446L79.85116279069767,158.66666666666666L80.93023255813954,158.66666666666666L82.00930232558139,159.44444444444446L83.08837209302325,158.66666666666666L84.16744186046512,159.44444444444446L85.24651162790697,159.44444444444446L86.32558139534883,159.44444444444446L87.4046511627907,160.22222222222223L88.48372093023255,158.66666666666666L89.5627906976744,159.44444444444446L90.64186046511628,160.22222222222223L91.72093023255815,160.22222222222223L92.8,159.44444444444446L93.87906976744185,159.44444444444446L94.95813953488371,159.44444444444446L96.03720930232559,159.44444444444446L97.11627906976744,159.44444444444446L98.1953488372093,160.22222222222223L99.27441860465116,159.44444444444446L100.35348837209303,159.44444444444446L101.43255813953488,160.22222222222223L102.51162790697673,159.44444444444446L103.5906976744186,159.44444444444446L104.66976744186047,158.66666666666666L105.74883720930232,158.66666666666666L106.82790697674417,158.66666666666666L107.90697674418604,159.44444444444446L108.98604651162792,159.44444444444446L110.06511627906977,159.44444444444446L111.14418604651162,158.66666666666666L112.22325581395349,158.66666666666666L113.30232558139535,158.66666666666666L114.3813953488372,159.44444444444446L115.46046511627905,159.44444444444446L116.53953488372093,159.44444444444446L117.6186046511628,159.44444444444446L118.69767441860465,159.44444444444446L119.7767441860465,159.44444444444446L120.85581395348837,159.44444444444446L121.93488372093024,158.66666666666666L123.0139534883721,157.88888888888889L124.09302325581395,157.88888888888889L125.17209302325581,158.66666666666666L126.25116279069768,159.44444444444446L127.33023255813953,159.44444444444446L128.4093023255814,159.44444444444446L129.48837209302326,159.44444444444446L130.56744186046512,158.66666666666666L131.646511627907,158.66666666666666L132.72558139534883,158.66666666666666L133.8046511627907,159.44444444444446L134.88372093023256,159.44444444444446L135.96279069767442,159.44444444444446L137.04186046511626,158.66666666666666L138.12093023255812,158.66666666666666L139.20000000000002,158.66666666666666L140.27906976744185,157.88888888888889L141.35813953488372,157.88888888888889L142.43720930232558,157.88888888888889L143.51627906976745,157.88888888888889L144.5953488372093,157.88888888888889L145.67441860465115,157.88888888888889L146.75348837209302,157.11111111111111L147.83255813953488,156.33333333333331L148.91162790697675,156.33333333333331L149.99069767441858,155.55555555555554L151.06976744186045,156.33333333333331L152.14883720930234,156.33333333333331L153.22790697674418,155.55555555555554L154.30697674418604,155.55555555555554L155.3860465116279,155.55555555555554L156.46511627906978,156.33333333333331L157.5441860465116,156.33333333333331L158.62325581395348,155.55555555555554L159.70232558139534,156.33333333333331L160.7813953488372,156.33333333333331L161.86046511627907,155.55555555555554L162.9395348837209,155.55555555555554L164.01860465116278,154L165.09767441860467,156.33333333333331L166.1767441860465,155.55555555555554L167.25581395348837,155.55555555555554L168.33488372093024,156.33333333333331L169.4139534883721,156.33333333333331L170.49302325581394,156.33333333333331L171.5720930232558,155.55555555555554L172.65116279069767,155.55555555555554L173.73023255813953,155.55555555555554L174.8093023255814,156.33333333333331L175.88837209302324,155.55555555555554L176.9674418604651,154.77777777777777L178.046511627907,154.77777777777777L179.1255813953488,154L180.2046511627907,154.77777777777777L181.28372093023256,154L182.36279069767443,154.77777777777777L183.4418604651163,154L184.52093023255813,154.77777777777777L185.6,154L186.67906976744186,154.77777777777777L187.7581395348837,154.77777777777777L188.83720930232556,154L189.91627906976743,154L190.99534883720932,154L192.07441860465119,154L193.15348837209302,154L194.2325581395349,154L195.31162790697675,154L196.3906976744186,154L197.46976744186045,154.77777777777777L198.54883720930232,155.55555555555554L199.62790697674419,154L200.70697674418605,154L201.7860465116279,154L202.86511627906975,153.22222222222223L203.94418604651165,154L205.02325581395345,153.22222222222223L206.10232558139535,154L207.1813953488372,154L208.26046511627908,154L209.33953488372094,154L210.41860465116278,154L211.49767441860465,155.55555555555554L212.5767441860465,155.55555555555554L213.65581395348835,155.55555555555554L214.7348837209302,155.55555555555554L215.81395348837208,155.55555555555554L216.89302325581397,154L217.97209302325584,154L219.05116279069767,154L220.13023255813954,154L221.2093023255814,154L222.28837209302324,154.77777777777777L223.3674418604651,154L224.44651162790697,154L225.52558139534884,154.77777777777777L226.6046511627907,154L227.68372093023254,154L228.7627906976744,154L229.8418604651163,154L230.9209302325581,154.77777777777777L232,154L233.07906976744187,154.77777777777777L234.15813953488373,155.55555555555554L235.2372093023256,154.77777777777777L236.31627906976743,155.55555555555554L237.3953488372093,155.55555555555554L238.47441860465116,155.55555555555554L239.553488372093,155.55555555555554L240.63255813953486,155.55555555555554L241.71162790697673,154.77777777777777L242.79069767441862,154.77777777777777L243.8697674418605,155.55555555555554L244.94883720930233,155.55555555555554L246.0279069767442,155.55555555555554L247.10697674418606,155.55555555555554L248.1860465116279,155.55555555555554L249.26511627906976,155.55555555555554L250.34418604651162,156.33333333333331L251.4232558139535,155.55555555555554L252.50232558139535,155.55555555555554L253.5813953488372,155.55555555555554L254.66046511627906,155.55555555555554L255.73953488372095,156.33333333333331L256.8186046511628,155.55555555555554L257.89767441860465,155.55555555555554L258.9767441860465,155.55555555555554L260.0558139534884,155.55555555555554L261.13488372093025,155.55555555555554L262.21395348837206,154.77777777777777L263.293023255814,154L264.37209302325584,154.77777777777777L265.45116279069765,155.55555555555554L266.5302325581395,155.55555555555554L267.6093023255814,155.55555555555554L268.68837209302325,155.55555555555554L269.7674418604651,155.55555555555554L270.846511627907,155.55555555555554L271.92558139534884,156.33333333333331L273.0046511627907,155.55555555555554L274.0837209302325,156.33333333333331L275.1627906976744,156.33333333333331L276.24186046511625,156.33333333333331L277.32093023255817,155.55555555555554L278.40000000000003,155.55555555555554L279.47906976744184,155.55555555555554L280.5581395348837,155.55555555555554L281.6372093023256,155.55555555555554L282.71627906976744,156.33333333333331L283.7953488372093,155.55555555555554L284.87441860465117,155.55555555555554L285.95348837209303,155.55555555555554L287.0325581395349,155.55555555555554L288.1116279069767,155.55555555555554L289.1906976744186,155.55555555555554L290.2697674418605,155.55555555555554L291.3488372093023,155.55555555555554L292.42790697674417,156.33333333333331L293.50697674418603,156.33333333333331L294.5860465116279,156.33333333333331L295.66511627906976,156.33333333333331L296.74418604651163,155.55555555555554L297.8232558139535,155.55555555555554L298.90232558139536,155.55555555555554L299.98139534883717,155.55555555555554L301.06046511627903,155.55555555555554L302.1395348837209,155.55555555555554L303.2186046511628,155.55555555555554L304.2976744186047,155.55555555555554L305.3767441860465,155.55555555555554L306.45581395348836,155.55555555555554L307.5348837209302,155.55555555555554L308.6139534883721,155.55555555555554L309.69302325581396,155.55555555555554L310.7720930232558,155.55555555555554L311.8511627906977,155.55555555555554L312.93023255813955,155.55555555555554L314.00930232558136,154.77777777777777L315.0883720930232,155.55555555555554L316.16744186046515,154.77777777777777L317.24651162790695,154.77777777777777L318.3255813953488,155.55555555555554L319.4046511627907,154.77777777777777L320.48372093023255,154L321.5627906976744,154L322.6418604651163,154L323.72093023255815,154L324.8,154.77777777777777L325.8790697674418,155.55555555555554L326.9581395348837,155.55555555555554L328.03720930232555,156.33333333333331L329.1162790697675,155.55555555555554L330.19534883720934,155.55555555555554L331.27441860465115,155.55555555555554L332.353488372093,155.55555555555554L333.4325581395349,155.55555555555554L334.51162790697674,155.55555555555554L335.5906976744186,155.55555555555554L336.6697674418605,155.55555555555554L337.74883720930234,155.55555555555554L338.8279069767442,155.55555555555554L339.906976744186,155.55555555555554L340.9860465116279,155.55555555555554L342.0651162790698,155.55555555555554L343.1441860465116,155.55555555555554L344.2232558139535,156.33333333333331L345.30232558139534,155.55555555555554L346.3813953488372,155.55555555555554L347.46046511627907,155.55555555555554L348.53953488372093,154L349.6186046511628,154.77777777777777L350.69767441860466,155.55555555555554L351.77674418604647,154.77777777777777L352.8558139534884,155.55555555555554L353.9348837209302,155.55555555555554L355.01395348837207,155.55555555555554L356.093023255814,154L357.1720930232558,154.77777777777777L358.2511627906976,154.77777777777777L359.3302325581395,154.77777777777777L360.4093023255814,154L361.48837209302326,154.77777777777777L362.5674418604651,154.77777777777777L363.64651162790693,154.77777777777777L364.72558139534885,154.77777777777777L365.80465116279066,154L366.8837209302326,154L367.96279069767445,153.22222222222223L369.04186046511626,154L370.1209302325582,154.77777777777777L371.2,154.77777777777777L372.27906976744185,155.55555555555554L373.3581395348837,154.77777777777777L374.4372093023256,155.55555555555554L375.5162790697674,155.55555555555554L376.5953488372093,155.55555555555554L377.6744186046511,155.55555555555554L378.75348837209305,155.55555555555554L379.83255813953485,155.55555555555554L380.9116279069767,154.77777777777777L381.99069767441864,155.55555555555554L383.06976744186045,155.55555555555554L384.14883720930237,155.55555555555554L385.2279069767442,155.55555555555554L386.30697674418604,156.33333333333331L387.3860465116279,155.55555555555554L388.4651162790698,156.33333333333331L389.5441860465116,155.55555555555554L390.6232558139535,155.55555555555554L391.7023255813953,155.55555555555554L392.7813953488372,155.55555555555554L393.8604651162791,155.55555555555554L394.9395348837209,156.33333333333331L396.01860465116283,155.55555555555554L397.09767441860464,156.33333333333331L398.17674418604645,156.33333333333331L399.25581395348837,156.33333333333331L400.33488372093024,156.33333333333331L401.4139534883721,155.55555555555554L402.49302325581397,155.55555555555554L403.5720930232558,156.33333333333331L404.6511627906977,157.11111111111111L405.7302325581395,157.88888888888889L406.80930232558137,157.11111111111111L407.8883720930233,156.33333333333331L408.9674418604651,156.33333333333331L410.0465116279069,155.55555555555554L411.12558139534883,156.33333333333331L412.2046511627907,157.11111111111111L413.28372093023256,155.55555555555554L414.3627906976744,157.11111111111111L415.44186046511624,156.33333333333331L416.52093023255816,157.11111111111111L417.59999999999997,155.55555555555554L418.6790697674419,155.55555555555554L419.75813953488375,157.11111111111111L420.83720930232556,155.55555555555554L421.9162790697675,156.33333333333331L422.9953488372093,156.33333333333331L424.0744186046511,155.55555555555554L425.153488372093,155.55555555555554L426.2325581395349,156.33333333333331L427.3116279069767,157.11111111111111L428.3906976744186,156.33333333333331L429.4697674418604,156.33333333333331L430.54883720930235,155.55555555555554L431.62790697674416,156.33333333333331L432.706976744186,156.33333333333331L433.78604651162794,155.55555555555554L434.86511627906975,156.33333333333331L435.9441860465117,156.33333333333331L437.0232558139535,156.33333333333331L438.10232558139535,156.33333333333331L439.1813953488372,156.33333333333331L440.2604651162791,156.33333333333331L441.3395348837209,156.33333333333331L442.4186046511628,156.33333333333331L443.4976744186046,156.33333333333331L444.5767441860465,156.33333333333331L445.65581395348835,156.33333333333331L446.7348837209302,155.55555555555554L447.81395348837214,156.33333333333331L448.89302325581394,156.33333333333331L449.97209302325575,156.33333333333331L451.0511627906977,156.33333333333331L452.13023255813954,156.33333333333331L453.2093023255814,156.33333333333331L454.28837209302327,156.33333333333331L455.3674418604651,156.33333333333331L456.446511627907,156.33333333333331L457.5255813953488,156.33333333333331L458.6046511627907,156.33333333333331L459.6837209302326,156.33333333333331L460.7627906976744,156.33333333333331L461.8418604651162,155.55555555555554L462.92093023255813,156.33333333333331L464,156.33333333333331L465.07906976744187,155.55555555555554L466.15813953488373,155.55555555555554L467.23720930232554,155.55555555555554L468.31627906976746,156.33333333333331L469.39534883720927,155.55555555555554L470.4744186046512,155.55555555555554L471.553488372093,155.55555555555554L472.63255813953486,156.33333333333331L473.7116279069768,156.33333333333331L474.7906976744186,155.55555555555554L475.8697674418604,155.55555555555554L476.9488372093023,155.55555555555554L478.0279069767442,155.55555555555554L479.106976744186,154.77777777777777L480.1860465116279,155.55555555555554L481.26511627906973,155.55555555555554L482.34418604651165,154.77777777777777L483.42325581395346,154.77777777777777L484.5023255813953,154.77777777777777L485.58139534883725,155.55555555555554L486.66046511627906,155.55555555555554L487.739534883721,154.77777777777777L488.8186046511628,154L489.89767441860465,154.77777777777777L490.9767441860465,155.55555555555554L492.0558139534884,155.55555555555554L493.1348837209302,154L494.2139534883721,154.77777777777777L495.2930232558139,154L496.3720930232558,154L497.45116279069765,155.55555555555554L498.5302325581395,154.77777777777777L499.60930232558144,154.77777777777777L500.68837209302325,155.55555555555554L501.76744186046506,155.55555555555554L502.846511627907,155.55555555555554L503.92558139534884,155.55555555555554L505.0046511627907,156.33333333333331L506.0837209302326,156.33333333333331L507.1627906976744,156.33333333333331L508.2418604651163,156.33333333333331L509.3209302325581,156.33333333333331L510.4,156.33333333333331L511.4790697674419,156.33333333333331L512.5581395348837,156.33333333333331L513.6372093023256,156.33333333333331L514.7162790697674,157.11111111111111L515.7953488372093,156.33333333333331L516.8744186046512,156.33333333333331L517.953488372093,156.33333333333331L519.0325581395349,157.11111111111111L520.1116279069768,156.33333333333331L521.1906976744186,156.33333333333331L522.2697674418605,156.33333333333331L523.3488372093024,157.11111111111111L524.4279069767441,157.11111111111111L525.5069767441861,156.33333333333331L526.586046511628,157.11111111111111L527.6651162790697,157.11111111111111L528.7441860465117,156.33333333333331L529.8232558139534,156.33333333333331L530.9023255813953,157.11111111111111L531.9813953488372,156.33333333333331L533.060465116279,156.33333333333331L534.1395348837209,156.33333333333331L535.2186046511628,155.55555555555554L536.2976744186046,156.33333333333331L537.3767441860465,155.55555555555554L538.4558139534884,155.55555555555554L539.5348837209302,155.55555555555554L540.6139534883721,155.55555555555554L541.693023255814,155.55555555555554L542.7720930232558,155.55555555555554L543.8511627906977,155.55555555555554L544.9302325581396,154.77777777777777L546.0093023255814,155.55555555555554L547.0883720930233,155.55555555555554L548.167441860465,155.55555555555554L549.246511627907,155.55555555555554L550.3255813953488,155.55555555555554L551.4046511627907,154L552.4837209302325,155.55555555555554L553.5627906976744,155.55555555555554L554.6418604651163,155.55555555555554L555.7209302325581,155.55555555555554L556.8000000000001,156.33333333333331L557.8790697674418,155.55555555555554L558.9581395348837,155.55555555555554L560.0372093023256,156.33333333333331L561.1162790697674,155.55555555555554L562.1953488372093,156.33333333333331L563.2744186046511,156.33333333333331L564.353488372093,156.33333333333331L565.4325581395349,156.33333333333331L566.5116279069767,156.33333333333331L567.5906976744186,156.33333333333331L568.6697674418605,155.55555555555554L569.7488372093023,156.33333333333331L570.8279069767442,157.88888888888889L571.9069767441861,157.11111111111111L572.9860465116279,157.11111111111111L574.0651162790698,156.33333333333331L575.1441860465117,157.88888888888889L576.2232558139534,157.88888888888889L577.3023255813954,157.88888888888889L578.3813953488371,157.88888888888889L579.460465116279,157.88888888888889L580.539534883721,157.88888888888889L581.6186046511627,157.11111111111111L582.6976744186046,157.11111111111111L583.7767441860465,157.11111111111111L584.8558139534883,157.11111111111111L585.9348837209302,157.88888888888889L587.0139534883721,157.11111111111111L588.0930232558139,157.11111111111111L589.1720930232558,157.11111111111111L590.2511627906977,157.11111111111111L591.3302325581395,145.44444444444446L592.4093023255814,145.44444444444446L593.4883720930233,157.11111111111111L594.5674418604651,157.11111111111111L595.646511627907,157.11111111111111L596.7255813953489,156.33333333333331L597.8046511627907,156.33333333333331L598.8837209302326,156.33333333333331L599.9627906976743,156.33333333333331L601.0418604651163,156.33333333333331L602.1209302325581,156.33333333333331L603.2,155.55555555555554L604.2790697674418,156.33333333333331L605.3581395348837,156.33333333333331L606.4372093023256,155.55555555555554L607.5162790697674,155.55555555555554L608.5953488372094,155.55555555555554L609.6744186046511,156.33333333333331L610.753488372093,156.33333333333331L611.8325581395349,156.33333333333331L612.9116279069767,156.33333333333331L613.9906976744186,156.33333333333331L615.0697674418604,156.33333333333331L616.1488372093023,156.33333333333331L617.2279069767442,156.33333333333331L618.306976744186,156.33333333333331L619.3860465116279,157.11111111111111L620.4651162790698,157.11111111111111L621.5441860465116,157.11111111111111L622.6232558139535,157.88888888888889L623.7023255813954,157.11111111111111L624.7813953488372,157.88888888888889L625.8604651162791,157.88888888888889L626.939534883721,157.11111111111111L628.0186046511627,157.88888888888889L629.0976744186047,157.88888888888889L630.1767441860464,157.88888888888889L631.2558139534883,157.88888888888889L632.3348837209303,157.88888888888889L633.413953488372,157.88888888888889L634.4930232558139,157.88888888888889L635.5720930232558,157.11111111111111L636.6511627906976,156.33333333333331L637.7302325581395,156.33333333333331L638.8093023255814,156.33333333333331L639.8883720930232,156.33333333333331L640.9674418604651,156.33333333333331L642.046511627907,157.11111111111111L643.1255813953488,157.11111111111111L644.2046511627907,156.33333333333331L645.2837209302326,157.11111111111111L646.3627906976744,156.33333333333331L647.4418604651163,156.33333333333331L648.5209302325582,157.11111111111111L649.6,156.33333333333331L650.6790697674419,156.33333333333331L651.7581395348836,156.33333333333331L652.8372093023256,156.33333333333331L653.9162790697674,156.33333333333331L654.9953488372093,155.55555555555554L656.0744186046511,155.55555555555554L657.153488372093,154.77777777777777L658.232558139535,155.55555555555554L659.3116279069767,154L660.3906976744187,154L661.4697674418604,154.77777777777777L662.5488372093023,154L663.6279069767442,154L664.706976744186,154L665.7860465116279,154L666.8651162790698,155.55555555555554L667.9441860465116,154L669.0232558139535,154L670.1023255813953,155.55555555555554L671.1813953488372,154L672.2604651162791,154L673.339534883721,154L674.4186046511627,154L675.4976744186047,154L676.5767441860465,153.22222222222223L677.6558139534884,153.22222222222223L678.7348837209303,152.44444444444443L679.813953488372,153.22222222222223L680.893023255814,151.66666666666666L681.9720930232558,153.22222222222223L683.0511627906976,152.44444444444443L684.1302325581396,152.44444444444443L685.2093023255813,153.22222222222223L686.2883720930232,153.22222222222223L687.3674418604651,154L688.446511627907,154.77777777777777L689.5255813953488,154L690.6046511627907,153.22222222222223L691.6837209302325,153.22222222222223L692.7627906976744,153.22222222222223L693.8418604651163,152.44444444444443L694.9209302325581,152.44444444444443L696,152.44444444444443"></path></g><g id="cadence" clip-path="url(#clip74)"><path fill="none" stroke-width="1.5px" class="simple-line" d="M0,231L1.0790697674418603,225.9090909090909L2.1581395348837207,230.36363636363637L3.2372093023255815,228.45454545454544L4.316279069767441,226.54545454545456L5.395348837209302,233.54545454545456L6.474418604651163,232.9090909090909L7.553488372093023,233.54545454545456L8.632558139534883,232.9090909090909L9.711627906976744,233.54545454545456L10.790697674418604,233.54545454545456L11.869767441860464,235.45454545454544L12.948837209302326,236.0909090909091L14.027906976744186,238L15.106976744186046,238L16.186046511627907,230.36363636363637L17.265116279069765,231L18.344186046511627,231.63636363636363L19.42325581395349,231.63636363636363L20.502325581395347,233.54545454545456L21.58139534883721,232.27272727272728L22.66046511627907,231L23.73953488372093,230.36363636363637L24.81860465116279,231L25.89767441860465,236.72727272727272L26.97674418604651,237.36363636363637L28.05581395348837,236.0909090909091L29.134883720930233,236.0909090909091L30.21395348837209,236.72727272727272L31.293023255813953,228.45454545454544L32.372093023255815,237.36363636363637L33.45116279069767,237.36363636363637L34.53023255813953,238L35.609302325581396,238L36.688372093023254,238.63636363636363L37.76744186046511,227.8181818181818L38.84651162790698,238.63636363636363L39.925581395348836,238.63636363636363L41.004651162790694,238L42.08372093023256,238L43.16279069767442,237.36363636363637L44.241860465116275,229.72727272727272L45.32093023255814,239.27272727272728L46.4,239.27272727272728L47.47906976744186,238.63636363636363L48.55813953488372,239.27272727272728L49.63720930232558,239.9090909090909L50.71627906976744,233.54545454545456L51.7953488372093,235.45454545454544L52.87441860465116,237.36363636363637L53.95348837209302,231.63636363636363L55.032558139534885,233.54545454545456L56.11162790697674,235.45454545454544L57.1906976744186,236.0909090909091L58.269767441860466,236.72727272727272L59.348837209302324,236.0909090909091L60.42790697674418,236.72727272727272L61.50697674418605,237.36363636363637L62.586046511627906,237.36363636363637L63.665116279069764,236.0909090909091L64.74418604651163,235.45454545454544L65.8232558139535,236.72727272727272L66.90232558139535,238.63636363636363L67.98139534883721,234.8181818181818L69.06046511627906,237.36363636363637L70.13953488372093,237.36363636363637L71.21860465116279,236.72727272727272L72.29767441860464,236.0909090909091L73.37674418604651,236.72727272727272L74.45581395348837,234.8181818181818L75.53488372093022,234.1818181818182L76.61395348837209,235.45454545454544L77.69302325581396,237.36363636363637L78.7720930232558,237.36363636363637L79.85116279069767,236.0909090909091L80.93023255813954,234.8181818181818L82.00930232558139,234.8181818181818L83.08837209302325,233.54545454545456L84.16744186046512,234.1818181818182L85.24651162790697,234.1818181818182L86.32558139534883,233.54545454545456L87.4046511627907,232.27272727272728L88.48372093023255,238.63636363636363L89.5627906976744,238.63636363636363L90.64186046511628,238L91.72093023255815,230.36363636363637L92.8,233.54545454545456L93.87906976744185,234.8181818181818L94.95813953488371,234.1818181818182L96.03720930232559,234.8181818181818L97.11627906976744,236.0909090909091L98.1953488372093,236.0909090909091L99.27441860465116,234.8181818181818L100.35348837209303,232.9090909090909L101.43255813953488,238.63636363636363L102.51162790697673,238L103.5906976744186,236.72727272727272L104.66976744186047,236.0909090909091L105.74883720930232,236.0909090909091L106.82790697674417,236.0909090909091L107.90697674418604,235.45454545454544L108.98604651162792,237.36363636363637L110.06511627906977,238L111.14418604651162,238L112.22325581395349,231.63636363636363L113.30232558139535,233.54545454545456L114.3813953488372,234.1818181818182L115.46046511627905,234.8181818181818L116.53953488372093,235.45454545454544L117.6186046511628,235.45454545454544L118.69767441860465,235.45454545454544L119.7767441860465,236.0909090909091L120.85581395348837,234.8181818181818L121.93488372093024,234.1818181818182L123.0139534883721,234.8181818181818L124.09302325581395,235.45454545454544L125.17209302325581,232.9090909090909L126.25116279069768,234.1818181818182L127.33023255813953,232.9090909090909L128.4093023255814,232.9090909090909L129.48837209302326,237.36363636363637L130.56744186046512,236.72727272727272L131.646511627907,231.63636363636363L132.72558139534883,231.63636363636363L133.8046511627907,232.9090909090909L134.88372093023256,239.27272727272728L135.96279069767442,236.0909090909091L137.04186046511626,234.8181818181818L138.12093023255812,234.8181818181818L139.20000000000002,233.54545454545456L140.27906976744185,231L141.35813953488372,232.9090909090909L142.43720930232558,234.1818181818182L143.51627906976745,234.1818181818182L144.5953488372093,236.0909090909091L145.67441860465115,229.0909090909091L146.75348837209302,232.9090909090909L147.83255813953488,231.63636363636363L148.91162790697675,233.54545454545456L149.99069767441858,235.45454545454544L151.06976744186045,234.1818181818182L152.14883720930234,234.8181818181818L153.22790697674418,234.1818181818182L154.30697674418604,233.54545454545456L155.3860465116279,231L156.46511627906978,239.27272727272728L157.5441860465116,235.45454545454544L158.62325581395348,234.1818181818182L159.70232558139534,234.1818181818182L160.7813953488372,236.0909090909091L161.86046511627907,235.45454545454544L162.9395348837209,234.1818181818182L164.01860465116278,236.72727272727272L165.09767441860467,237.36363636363637L166.1767441860465,238L167.25581395348837,230.36363636363637L168.33488372093024,239.27272727272728L169.4139534883721,239.9090909090909L170.49302325581394,229.72727272727272L171.5720930232558,240.54545454545456L172.65116279069767,234.1818181818182L173.73023255813953,236.0909090909091L174.8093023255814,236.0909090909091L175.88837209302324,237.36363636363637L176.9674418604651,237.36363636363637L178.046511627907,236.72727272727272L179.1255813953488,236.0909090909091L180.2046511627907,235.45454545454544L181.28372093023256,239.27272727272728L182.36279069767443,236.72727272727272L183.4418604651163,233.54545454545456L184.52093023255813,232.27272727272728L185.6,234.1818181818182L186.67906976744186,228.45454545454544L187.7581395348837,234.1818181818182L188.83720930232556,236.72727272727272L189.91627906976743,236.0909090909091L190.99534883720932,235.45454545454544L192.07441860465119,234.8181818181818L193.15348837209302,238.63636363636363L194.2325581395349,231.63636363636363L195.31162790697675,233.54545454545456L196.3906976744186,232.9090909090909L197.46976744186045,234.8181818181818L198.54883720930232,237.36363636363637L199.62790697674419,235.45454545454544L200.70697674418605,233.54545454545456L201.7860465116279,233.54545454545456L202.86511627906975,231.63636363636363L203.94418604651165,231.63636363636363L205.02325581395345,235.45454545454544L206.10232558139535,235.45454545454544L207.1813953488372,234.8181818181818L208.26046511627908,236.0909090909091L209.33953488372094,234.1818181818182L210.41860465116278,232.27272727272728L211.49767441860465,231.63636363636363L212.5767441860465,232.27272727272728L213.65581395348835,234.8181818181818L214.7348837209302,237.36363636363637L215.81395348837208,232.27272727272728L216.89302325581397,234.8181818181818L217.97209302325584,235.45454545454544L219.05116279069767,234.1818181818182L220.13023255813954,231.63636363636363L221.2093023255814,232.27272727272728L222.28837209302324,232.9090909090909L223.3674418604651,232.9090909090909L224.44651162790697,233.54545454545456L225.52558139534884,234.1818181818182L226.6046511627907,232.27272727272728L227.68372093023254,233.54545454545456L228.7627906976744,232.9090909090909L229.8418604651163,232.9090909090909L230.9209302325581,234.8181818181818L232,232.9090909090909L233.07906976744187,233.54545454545456L234.15813953488373,234.1818181818182L235.2372093023256,236.0909090909091L236.31627906976743,236.0909090909091L237.3953488372093,231L238.47441860465116,232.9090909090909L239.553488372093,233.54545454545456L240.63255813953486,233.54545454545456L241.71162790697673,234.8181818181818L242.79069767441862,235.45454545454544L243.8697674418605,237.36363636363637L244.94883720930233,237.36363636363637L246.0279069767442,236.0909090909091L247.10697674418606,236.72727272727272L248.1860465116279,237.36363636363637L249.26511627906976,236.72727272727272L250.34418604651162,237.36363636363637L251.4232558139535,237.36363636363637L252.50232558139535,235.45454545454544L253.5813953488372,234.8181818181818L254.66046511627906,234.8181818181818L255.73953488372095,236.0909090909091L256.8186046511628,232.27272727272728L257.89767441860465,235.45454545454544L258.9767441860465,237.36363636363637L260.0558139534884,237.36363636363637L261.13488372093025,238L262.21395348837206,236.0909090909091L263.293023255814,235.45454545454544L264.37209302325584,234.1818181818182L265.45116279069765,234.1818181818182L266.5302325581395,232.9090909090909L267.6093023255814,236.72727272727272L268.68837209302325,234.8181818181818L269.7674418604651,233.54545454545456L270.846511627907,234.1818181818182L271.92558139534884,236.0909090909091L273.0046511627907,234.1818181818182L274.0837209302325,232.27272727272728L275.1627906976744,236.72727272727272L276.24186046511625,236.72727272727272L277.32093023255817,238.63636363636363L278.40000000000003,234.1818181818182L279.47906976744184,234.1818181818182L280.5581395348837,232.27272727272728L281.6372093023256,234.1818181818182L282.71627906976744,234.1818181818182L283.7953488372093,235.45454545454544L284.87441860465117,234.1818181818182L285.95348837209303,233.54545454545456L287.0325581395349,232.27272727272728L288.1116279069767,239.27272727272728L289.1906976744186,237.36363636363637L290.2697674418605,236.0909090909091L291.3488372093023,235.45454545454544L292.42790697674417,236.0909090909091L293.50697674418603,236.0909090909091L294.5860465116279,235.45454545454544L295.66511627906976,235.45454545454544L296.74418604651163,235.45454545454544L297.8232558139535,234.8181818181818L298.90232558139536,234.1818181818182L299.98139534883717,232.27272727272728L301.06046511627903,232.9090909090909L302.1395348837209,234.1818181818182L303.2186046511628,230.36363636363637L304.2976744186047,230.36363636363637L305.3767441860465,234.8181818181818L306.45581395348836,230.36363636363637L307.5348837209302,232.27272727272728L308.6139534883721,231L309.69302325581396,237.36363636363637L310.7720930232558,233.54545454545456L311.8511627906977,231.63636363636363L312.93023255813955,231L314.00930232558136,231L315.0883720930232,232.27272727272728L316.16744186046515,234.1818181818182L317.24651162790695,236.72727272727272L318.3255813953488,237.36363636363637L319.4046511627907,239.27272727272728L320.48372093023255,231.63636363636363L321.5627906976744,233.54545454545456L322.6418604651163,234.1818181818182L323.72093023255815,234.1818181818182L324.8,233.54545454545456L325.8790697674418,239.9090909090909L326.9581395348837,236.72727272727272L328.03720930232555,235.45454545454544L329.1162790697675,236.0909090909091L330.19534883720934,236.0909090909091L331.27441860465115,234.8181818181818L332.353488372093,236.0909090909091L333.4325581395349,236.0909090909091L334.51162790697674,236.72727272727272L335.5906976744186,237.36363636363637L336.6697674418605,237.36363636363637L337.74883720930234,230.36363636363637L338.8279069767442,234.8181818181818L339.906976744186,236.0909090909091L340.9860465116279,233.54545454545456L342.0651162790698,232.9090909090909L343.1441860465116,234.8181818181818L344.2232558139535,235.45454545454544L345.30232558139534,235.45454545454544L346.3813953488372,236.72727272727272L347.46046511627907,236.72727272727272L348.53953488372093,237.36363636363637L349.6186046511628,237.36363636363637L350.69767441860466,237.36363636363637L351.77674418604647,235.45454545454544L352.8558139534884,236.72727272727272L353.9348837209302,237.36363636363637L355.01395348837207,236.0909090909091L356.093023255814,237.36363636363637L357.1720930232558,237.36363636363637L358.2511627906976,237.36363636363637L359.3302325581395,236.0909090909091L360.4093023255814,236.0909090909091L361.48837209302326,235.45454545454544L362.5674418604651,236.72727272727272L363.64651162790693,236.0909090909091L364.72558139534885,234.1818181818182L365.80465116279066,232.9090909090909L366.8837209302326,235.45454545454544L367.96279069767445,232.9090909090909L369.04186046511626,232.9090909090909L370.1209302325582,234.8181818181818L371.2,234.1818181818182L372.27906976744185,231L373.3581395348837,234.1818181818182L374.4372093023256,236.0909090909091L375.5162790697674,236.0909090909091L376.5953488372093,235.45454545454544L377.6744186046511,236.0909090909091L378.75348837209305,231L379.83255813953485,231.63636363636363L380.9116279069767,233.54545454545456L381.99069767441864,234.1818181818182L383.06976744186045,238L384.14883720930237,236.72727272727272L385.2279069767442,236.72727272727272L386.30697674418604,235.45454545454544L387.3860465116279,236.72727272727272L388.4651162790698,230.36363636363637L389.5441860465116,236.0909090909091L390.6232558139535,229.72727272727272L391.7023255813953,233.54545454545456L392.7813953488372,235.45454545454544L393.8604651162791,234.8181818181818L394.9395348837209,237.36363636363637L396.01860465116283,234.8181818181818L397.09767441860464,233.54545454545456L398.17674418604645,236.0909090909091L399.25581395348837,239.27272727272728L400.33488372093024,236.72727272727272L401.4139534883721,239.27272727272728L402.49302325581397,276.8181818181818L403.5720930232558,249.45454545454547L404.6511627906977,222.72727272727275L405.7302325581395,231L406.80930232558137,227.8181818181818L407.8883720930233,228.45454545454544L408.9674418604651,227.8181818181818L410.0465116279069,229.72727272727272L411.12558139534883,232.27272727272728L412.2046511627907,230.36363636363637L413.28372093023256,232.27272727272728L414.3627906976744,234.8181818181818L415.44186046511624,232.27272727272728L416.52093023255816,232.27272727272728L417.59999999999997,232.27272727272728L418.6790697674419,231.63636363636363L419.75813953488375,230.36363636363637L420.83720930232556,232.9090909090909L421.9162790697675,231L422.9953488372093,230.36363636363637L424.0744186046511,230.36363636363637L425.153488372093,231L426.2325581395349,229.0909090909091L427.3116279069767,232.27272727272728L428.3906976744186,234.1818181818182L429.4697674418604,232.9090909090909L430.54883720930235,233.54545454545456L431.62790697674416,229.72727272727272L432.706976744186,232.27272727272728L433.78604651162794,232.27272727272728L434.86511627906975,234.1818181818182L435.9441860465117,234.8181818181818L437.0232558139535,236.72727272727272L438.10232558139535,234.1818181818182L439.1813953488372,233.54545454545456L440.2604651162791,232.9090909090909L441.3395348837209,231L442.4186046511628,231L443.4976744186046,229.0909090909091L444.5767441860465,236.0909090909091L445.65581395348835,234.8181818181818L446.7348837209302,234.8181818181818L447.81395348837214,232.27272727272728L448.89302325581394,238L449.97209302325575,232.9090909090909L451.0511627906977,230.36363636363637L452.13023255813954,232.27272727272728L453.2093023255814,233.54545454545456L454.28837209302327,229.72727272727272L455.3674418604651,232.9090909090909L456.446511627907,232.27272727272728L457.5255813953488,231.63636363636363L458.6046511627907,235.45454545454544L459.6837209302326,236.0909090909091L460.7627906976744,233.54545454545456L461.8418604651162,232.27272727272728L462.92093023255813,231L464,232.27272727272728L465.07906976744187,232.27272727272728L466.15813953488373,235.45454545454544L467.23720930232554,233.54545454545456L468.31627906976746,233.54545454545456L469.39534883720927,235.45454545454544L470.4744186046512,233.54545454545456L471.553488372093,232.9090909090909L472.63255813953486,235.45454545454544L473.7116279069768,234.8181818181818L474.7906976744186,237.36363636363637L475.8697674418604,232.9090909090909L476.9488372093023,233.54545454545456L478.0279069767442,231.63636363636363L479.106976744186,233.54545454545456L480.1860465116279,234.1818181818182L481.26511627906973,234.8181818181818L482.34418604651165,236.0909090909091L483.42325581395346,237.36363636363637L484.5023255813953,237.36363636363637L485.58139534883725,233.54545454545456L486.66046511627906,238L487.739534883721,236.72727272727272L488.8186046511628,237.36363636363637L489.89767441860465,235.45454545454544L490.9767441860465,236.0909090909091L492.0558139534884,236.0909090909091L493.1348837209302,236.0909090909091L494.2139534883721,234.8181818181818L495.2930232558139,232.9090909090909L496.3720930232558,231.63636363636363L497.45116279069765,231.63636363636363L498.5302325581395,234.1818181818182L499.60930232558144,230.36363636363637L500.68837209302325,231L501.76744186046506,232.27272727272728L502.846511627907,231L503.92558139534884,232.27272727272728L505.0046511627907,231L506.0837209302326,234.8181818181818L507.1627906976744,233.54545454545456L508.2418604651163,231L509.3209302325581,232.27272727272728L510.4,229.0909090909091L511.4790697674419,234.1818181818182L512.5581395348837,234.1818181818182L513.6372093023256,234.8181818181818L514.7162790697674,228.45454545454544L515.7953488372093,236.72727272727272L516.8744186046512,232.27272727272728L517.953488372093,233.54545454545456L519.0325581395349,232.9090909090909L520.1116279069768,234.8181818181818L521.1906976744186,233.54545454545456L522.2697674418605,232.9090909090909L523.3488372093024,232.9090909090909L524.4279069767441,232.27272727272728L525.5069767441861,232.9090909090909L526.586046511628,232.27272727272728L527.6651162790697,232.27272727272728L528.7441860465117,231.63636363636363L529.8232558139534,232.9090909090909L530.9023255813953,232.9090909090909L531.9813953488372,234.1818181818182L533.060465116279,234.8181818181818L534.1395348837209,235.45454545454544L535.2186046511628,235.45454545454544L536.2976744186046,234.1818181818182L537.3767441860465,234.1818181818182L538.4558139534884,232.9090909090909L539.5348837209302,234.1818181818182L540.6139534883721,233.54545454545456L541.693023255814,234.1818181818182L542.7720930232558,235.45454545454544L543.8511627906977,234.1818181818182L544.9302325581396,235.45454545454544L546.0093023255814,236.0909090909091L547.0883720930233,231L548.167441860465,233.54545454545456L549.246511627907,232.27272727272728L550.3255813953488,235.45454545454544L551.4046511627907,234.1818181818182L552.4837209302325,231.63636363636363L553.5627906976744,233.54545454545456L554.6418604651163,233.54545454545456L555.7209302325581,230.36363636363637L556.8000000000001,233.54545454545456L557.8790697674418,235.45454545454544L558.9581395348837,236.0909090909091L560.0372093023256,232.9090909090909L561.1162790697674,232.27272727272728L562.1953488372093,231L563.2744186046511,233.54545454545456L564.353488372093,234.1818181818182L565.4325581395349,234.1818181818182L566.5116279069767,234.1818181818182L567.5906976744186,232.27272727272728L568.6697674418605,233.54545454545456L569.7488372093023,232.9090909090909L570.8279069767442,235.45454545454544L571.9069767441861,236.0909090909091L572.9860465116279,231.63636363636363L574.0651162790698,236.0909090909091L575.1441860465117,234.8181818181818L576.2232558139534,232.27272727272728L577.3023255813954,230.36363636363637L578.3813953488371,235.45454545454544L579.460465116279,231.63636363636363L580.539534883721,233.54545454545456L581.6186046511627,235.45454545454544L582.6976744186046,235.45454545454544L583.7767441860465,236.0909090909091L584.8558139534883,234.1818181818182L585.9348837209302,232.9090909090909L587.0139534883721,232.27272727272728L588.0930232558139,233.54545454545456L589.1720930232558,235.45454545454544L590.2511627906977,236.0909090909091L591.3302325581395,236.0909090909091L592.4093023255814,236.0909090909091L593.4883720930233,232.9090909090909L594.5674418604651,236.0909090909091L595.646511627907,234.8181818181818L596.7255813953489,234.8181818181818L597.8046511627907,233.54545454545456L598.8837209302326,233.54545454545456L599.9627906976743,234.1818181818182L601.0418604651163,234.1818181818182L602.1209302325581,230.36363636363637L603.2,232.27272727272728L604.2790697674418,233.54545454545456L605.3581395348837,234.1818181818182L606.4372093023256,234.8181818181818L607.5162790697674,232.9090909090909L608.5953488372094,234.1818181818182L609.6744186046511,234.8181818181818L610.753488372093,233.54545454545456L611.8325581395349,235.45454545454544L612.9116279069767,234.1818181818182L613.9906976744186,234.8181818181818L615.0697674418604,234.8181818181818L616.1488372093023,237.36363636363637L617.2279069767442,232.27272727272728L618.306976744186,234.1818181818182L619.3860465116279,234.8181818181818L620.4651162790698,234.1818181818182L621.5441860465116,233.54545454545456L622.6232558139535,233.54545454545456L623.7023255813954,234.1818181818182L624.7813953488372,232.9090909090909L625.8604651162791,234.8181818181818L626.939534883721,232.9090909090909L628.0186046511627,232.27272727272728L629.0976744186047,228.45454545454544L630.1767441860464,231L631.2558139534883,231.63636363636363L632.3348837209303,232.27272727272728L633.413953488372,232.27272727272728L634.4930232558139,231.63636363636363L635.5720930232558,233.54545454545456L636.6511627906976,232.9090909090909L637.7302325581395,232.27272727272728L638.8093023255814,235.45454545454544L639.8883720930232,234.1818181818182L640.9674418604651,233.54545454545456L642.046511627907,232.9090909090909L643.1255813953488,234.1818181818182L644.2046511627907,233.54545454545456L645.2837209302326,232.27272727272728L646.3627906976744,232.27272727272728L647.4418604651163,237.36363636363637L648.5209302325582,234.1818181818182L649.6,234.8181818181818L650.6790697674419,233.54545454545456L651.7581395348836,234.1818181818182L652.8372093023256,234.1818181818182L653.9162790697674,234.8181818181818L654.9953488372093,232.27272727272728L656.0744186046511,233.54545454545456L657.153488372093,234.8181818181818L658.232558139535,235.45454545454544L659.3116279069767,234.8181818181818L660.3906976744187,234.1818181818182L661.4697674418604,235.45454545454544L662.5488372093023,235.45454545454544L663.6279069767442,234.1818181818182L664.706976744186,234.1818181818182L665.7860465116279,234.1818181818182L666.8651162790698,235.45454545454544L667.9441860465116,234.1818181818182L669.0232558139535,234.1818181818182L670.1023255813953,232.9090909090909L671.1813953488372,234.1818181818182L672.2604651162791,231L673.339534883721,234.1818181818182L674.4186046511627,235.45454545454544L675.4976744186047,236.0909090909091L676.5767441860465,236.72727272727272L677.6558139534884,233.54545454545456L678.7348837209303,232.9090909090909L679.813953488372,234.1818181818182L680.893023255814,233.54545454545456L681.9720930232558,233.54545454545456L683.0511627906976,232.27272727272728L684.1302325581396,231.63636363636363L685.2093023255813,210.63636363636363L686.2883720930232,225.9090909090909L687.3674418604651,229.72727272727272L688.446511627907,234.1818181818182L689.5255813953488,225.9090909090909L690.6046511627907,224L691.6837209302325,226.54545454545456L692.7627906976744,241.1818181818182L693.8418604651163,240.54545454545456L694.9209302325581,240.54545454545456L696,242.45454545454544"></path></g><g id="temp" clip-path="url(#clip79)"><path fill="none" stroke-width="1.5px" class="simple-line" d="M0,302.9090909090909L1.0790697674418603,302.9090909090909L2.1581395348837207,302.9090909090909L3.2372093023255815,302.9090909090909L4.316279069767441,302.9090909090909L5.395348837209302,302.9090909090909L6.474418604651163,302.9090909090909L7.553488372093023,302.9090909090909L8.632558139534883,302.9090909090909L9.711627906976744,302.9090909090909L10.790697674418604,302.9090909090909L11.869767441860464,302.9090909090909L12.948837209302326,302.9090909090909L14.027906976744186,302.9090909090909L15.106976744186046,302.9090909090909L16.186046511627907,302.9090909090909L17.265116279069765,302.9090909090909L18.344186046511627,302.9090909090909L19.42325581395349,302.9090909090909L20.502325581395347,302.9090909090909L21.58139534883721,302.9090909090909L22.66046511627907,302.9090909090909L23.73953488372093,302.9090909090909L24.81860465116279,302.9090909090909L25.89767441860465,302.9090909090909L26.97674418604651,302.9090909090909L28.05581395348837,302.9090909090909L29.134883720930233,302.9090909090909L30.21395348837209,302.9090909090909L31.293023255813953,302.9090909090909L32.372093023255815,302.9090909090909L33.45116279069767,302.9090909090909L34.53023255813953,302.9090909090909L35.609302325581396,302.9090909090909L36.688372093023254,302.9090909090909L37.76744186046511,302.9090909090909L38.84651162790698,302.9090909090909L39.925581395348836,302.9090909090909L41.004651162790694,302.9090909090909L42.08372093023256,302.9090909090909L43.16279069767442,302.9090909090909L44.241860465116275,302.9090909090909L45.32093023255814,302.9090909090909L46.4,302.9090909090909L47.47906976744186,302.9090909090909L48.55813953488372,302.9090909090909L49.63720930232558,302.9090909090909L50.71627906976744,302.9090909090909L51.7953488372093,302.9090909090909L52.87441860465116,302.9090909090909L53.95348837209302,302.9090909090909L55.032558139534885,302.9090909090909L56.11162790697674,302.9090909090909L57.1906976744186,302.9090909090909L58.269767441860466,302.9090909090909L59.348837209302324,302.9090909090909L60.42790697674418,302.9090909090909L61.50697674418605,302.9090909090909L62.586046511627906,302.9090909090909L63.665116279069764,302.9090909090909L64.74418604651163,302.9090909090909L65.8232558139535,302.9090909090909L66.90232558139535,302.9090909090909L67.98139534883721,302.9090909090909L69.06046511627906,302.9090909090909L70.13953488372093,302.9090909090909L71.21860465116279,302.9090909090909L72.29767441860464,302.9090909090909L73.37674418604651,302.9090909090909L74.45581395348837,302.9090909090909L75.53488372093022,302.9090909090909L76.61395348837209,302.9090909090909L77.69302325581396,302.9090909090909L78.7720930232558,302.9090909090909L79.85116279069767,302.9090909090909L80.93023255813954,302.9090909090909L82.00930232558139,302.9090909090909L83.08837209302325,302.9090909090909L84.16744186046512,302.9090909090909L85.24651162790697,302.9090909090909L86.32558139534883,302.9090909090909L87.4046511627907,302.9090909090909L88.48372093023255,302.9090909090909L89.5627906976744,302.9090909090909L90.64186046511628,302.9090909090909L91.72093023255815,302.9090909090909L92.8,302.9090909090909L93.87906976744185,302.9090909090909L94.95813953488371,302.9090909090909L96.03720930232559,302.9090909090909L97.11627906976744,302.9090909090909L98.1953488372093,302.9090909090909L99.27441860465116,302.9090909090909L100.35348837209303,302.9090909090909L101.43255813953488,302.9090909090909L102.51162790697673,302.9090909090909L103.5906976744186,302.9090909090909L104.66976744186047,302.9090909090909L105.74883720930232,302.9090909090909L106.82790697674417,302.9090909090909L107.90697674418604,302.9090909090909L108.98604651162792,302.9090909090909L110.06511627906977,302.9090909090909L111.14418604651162,302.9090909090909L112.22325581395349,302.9090909090909L113.30232558139535,302.9090909090909L114.3813953488372,302.9090909090909L115.46046511627905,302.9090909090909L116.53953488372093,302.9090909090909L117.6186046511628,302.9090909090909L118.69767441860465,302.9090909090909L119.7767441860465,302.9090909090909L120.85581395348837,302.9090909090909L121.93488372093024,302.9090909090909L123.0139534883721,302.9090909090909L124.09302325581395,302.9090909090909L125.17209302325581,302.9090909090909L126.25116279069768,302.9090909090909L127.33023255813953,302.9090909090909L128.4093023255814,302.9090909090909L129.48837209302326,302.9090909090909L130.56744186046512,302.9090909090909L131.646511627907,302.9090909090909L132.72558139534883,302.9090909090909L133.8046511627907,302.9090909090909L134.88372093023256,302.9090909090909L135.96279069767442,302.9090909090909L137.04186046511626,302.9090909090909L138.12093023255812,302.9090909090909L139.20000000000002,302.9090909090909L140.27906976744185,302.9090909090909L141.35813953488372,302.9090909090909L142.43720930232558,302.9090909090909L143.51627906976745,302.9090909090909L144.5953488372093,302.9090909090909L145.67441860465115,302.9090909090909L146.75348837209302,302.9090909090909L147.83255813953488,302.9090909090909L148.91162790697675,302.9090909090909L149.99069767441858,302.9090909090909L151.06976744186045,325.81818181818176L152.14883720930234,325.81818181818176L153.22790697674418,325.81818181818176L154.30697674418604,325.81818181818176L155.3860465116279,325.81818181818176L156.46511627906978,325.81818181818176L157.5441860465116,325.81818181818176L158.62325581395348,325.81818181818176L159.70232558139534,325.81818181818176L160.7813953488372,325.81818181818176L161.86046511627907,325.81818181818176L162.9395348837209,325.81818181818176L164.01860465116278,325.81818181818176L165.09767441860467,325.81818181818176L166.1767441860465,325.81818181818176L167.25581395348837,325.81818181818176L168.33488372093024,325.81818181818176L169.4139534883721,325.81818181818176L170.49302325581394,325.81818181818176L171.5720930232558,325.81818181818176L172.65116279069767,325.81818181818176L173.73023255813953,325.81818181818176L174.8093023255814,325.81818181818176L175.88837209302324,325.81818181818176L176.9674418604651,325.81818181818176L178.046511627907,325.81818181818176L179.1255813953488,325.81818181818176L180.2046511627907,325.81818181818176L181.28372093023256,325.81818181818176L182.36279069767443,325.81818181818176L183.4418604651163,325.81818181818176L184.52093023255813,325.81818181818176L185.6,325.81818181818176L186.67906976744186,325.81818181818176L187.7581395348837,325.81818181818176L188.83720930232556,325.81818181818176L189.91627906976743,325.81818181818176L190.99534883720932,325.81818181818176L192.07441860465119,325.81818181818176L193.15348837209302,325.81818181818176L194.2325581395349,325.81818181818176L195.31162790697675,325.81818181818176L196.3906976744186,325.81818181818176L197.46976744186045,325.81818181818176L198.54883720930232,325.81818181818176L199.62790697674419,325.81818181818176L200.70697674418605,325.81818181818176L201.7860465116279,325.81818181818176L202.86511627906975,325.81818181818176L203.94418604651165,325.81818181818176L205.02325581395345,325.81818181818176L206.10232558139535,325.81818181818176L207.1813953488372,325.81818181818176L208.26046511627908,325.81818181818176L209.33953488372094,325.81818181818176L210.41860465116278,325.81818181818176L211.49767441860465,325.81818181818176L212.5767441860465,325.81818181818176L213.65581395348835,325.81818181818176L214.7348837209302,325.81818181818176L215.81395348837208,325.81818181818176L216.89302325581397,325.81818181818176L217.97209302325584,325.81818181818176L219.05116279069767,325.81818181818176L220.13023255813954,325.81818181818176L221.2093023255814,325.81818181818176L222.28837209302324,325.81818181818176L223.3674418604651,325.81818181818176L224.44651162790697,325.81818181818176L225.52558139534884,325.81818181818176L226.6046511627907,325.81818181818176L227.68372093023254,325.81818181818176L228.7627906976744,325.81818181818176L229.8418604651163,325.81818181818176L230.9209302325581,325.81818181818176L232,325.81818181818176L233.07906976744187,325.81818181818176L234.15813953488373,325.81818181818176L235.2372093023256,325.81818181818176L236.31627906976743,325.81818181818176L237.3953488372093,325.81818181818176L238.47441860465116,325.81818181818176L239.553488372093,325.81818181818176L240.63255813953486,325.81818181818176L241.71162790697673,325.81818181818176L242.79069767441862,325.81818181818176L243.8697674418605,325.81818181818176L244.94883720930233,325.81818181818176L246.0279069767442,325.81818181818176L247.10697674418606,325.81818181818176L248.1860465116279,325.81818181818176L249.26511627906976,325.81818181818176L250.34418604651162,325.81818181818176L251.4232558139535,325.81818181818176L252.50232558139535,325.81818181818176L253.5813953488372,325.81818181818176L254.66046511627906,325.81818181818176L255.73953488372095,325.81818181818176L256.8186046511628,325.81818181818176L257.89767441860465,325.81818181818176L258.9767441860465,325.81818181818176L260.0558139534884,325.81818181818176L261.13488372093025,325.81818181818176L262.21395348837206,325.81818181818176L263.293023255814,325.81818181818176L264.37209302325584,325.81818181818176L265.45116279069765,325.81818181818176L266.5302325581395,325.81818181818176L267.6093023255814,325.81818181818176L268.68837209302325,325.81818181818176L269.7674418604651,325.81818181818176L270.846511627907,325.81818181818176L271.92558139534884,325.81818181818176L273.0046511627907,325.81818181818176L274.0837209302325,325.81818181818176L275.1627906976744,325.81818181818176L276.24186046511625,325.81818181818176L277.32093023255817,325.81818181818176L278.40000000000003,325.81818181818176L279.47906976744184,325.81818181818176L280.5581395348837,325.81818181818176L281.6372093023256,325.81818181818176L282.71627906976744,325.81818181818176L283.7953488372093,325.81818181818176L284.87441860465117,325.81818181818176L285.95348837209303,325.81818181818176L287.0325581395349,325.81818181818176L288.1116279069767,325.81818181818176L289.1906976744186,325.81818181818176L290.2697674418605,325.81818181818176L291.3488372093023,325.81818181818176L292.42790697674417,325.81818181818176L293.50697674418603,325.81818181818176L294.5860465116279,325.81818181818176L295.66511627906976,325.81818181818176L296.74418604651163,325.81818181818176L297.8232558139535,325.81818181818176L298.90232558139536,325.81818181818176L299.98139534883717,325.81818181818176L301.06046511627903,325.81818181818176L302.1395348837209,325.81818181818176L303.2186046511628,325.81818181818176L304.2976744186047,325.81818181818176L305.3767441860465,325.81818181818176L306.45581395348836,325.81818181818176L307.5348837209302,325.81818181818176L308.6139534883721,325.81818181818176L309.69302325581396,325.81818181818176L310.7720930232558,325.81818181818176L311.8511627906977,325.81818181818176L312.93023255813955,325.81818181818176L314.00930232558136,325.81818181818176L315.0883720930232,325.81818181818176L316.16744186046515,325.81818181818176L317.24651162790695,325.81818181818176L318.3255813953488,325.81818181818176L319.4046511627907,325.81818181818176L320.48372093023255,325.81818181818176L321.5627906976744,325.81818181818176L322.6418604651163,325.81818181818176L323.72093023255815,325.81818181818176L324.8,325.81818181818176L325.8790697674418,325.81818181818176L326.9581395348837,325.81818181818176L328.03720930232555,325.81818181818176L329.1162790697675,325.81818181818176L330.19534883720934,325.81818181818176L331.27441860465115,348.7272727272727L332.353488372093,348.7272727272727L333.4325581395349,348.7272727272727L334.51162790697674,348.7272727272727L335.5906976744186,348.7272727272727L336.6697674418605,348.7272727272727L337.74883720930234,348.7272727272727L338.8279069767442,348.7272727272727L339.906976744186,348.7272727272727L340.9860465116279,348.7272727272727L342.0651162790698,348.7272727272727L343.1441860465116,348.7272727272727L344.2232558139535,348.7272727272727L345.30232558139534,348.7272727272727L346.3813953488372,348.7272727272727L347.46046511627907,348.7272727272727L348.53953488372093,348.7272727272727L349.6186046511628,348.7272727272727L350.69767441860466,348.7272727272727L351.77674418604647,348.7272727272727L352.8558139534884,348.7272727272727L353.9348837209302,348.7272727272727L355.01395348837207,348.7272727272727L356.093023255814,348.7272727272727L357.1720930232558,348.7272727272727L358.2511627906976,348.7272727272727L359.3302325581395,348.7272727272727L360.4093023255814,348.7272727272727L361.48837209302326,348.7272727272727L362.5674418604651,348.7272727272727L363.64651162790693,348.7272727272727L364.72558139534885,348.7272727272727L365.80465116279066,348.7272727272727L366.8837209302326,348.7272727272727L367.96279069767445,348.7272727272727L369.04186046511626,348.7272727272727L370.1209302325582,348.7272727272727L371.2,348.7272727272727L372.27906976744185,348.7272727272727L373.3581395348837,348.7272727272727L374.4372093023256,348.7272727272727L375.5162790697674,348.7272727272727L376.5953488372093,348.7272727272727L377.6744186046511,348.7272727272727L378.75348837209305,348.7272727272727L379.83255813953485,348.7272727272727L380.9116279069767,348.7272727272727L381.99069767441864,348.7272727272727L383.06976744186045,348.7272727272727L384.14883720930237,348.7272727272727L385.2279069767442,348.7272727272727L386.30697674418604,348.7272727272727L387.3860465116279,348.7272727272727L388.4651162790698,348.7272727272727L389.5441860465116,348.7272727272727L390.6232558139535,348.7272727272727L391.7023255813953,348.7272727272727L392.7813953488372,348.7272727272727L393.8604651162791,348.7272727272727L394.9395348837209,348.7272727272727L396.01860465116283,348.7272727272727L397.09767441860464,348.7272727272727L398.17674418604645,348.7272727272727L399.25581395348837,348.7272727272727L400.33488372093024,348.7272727272727L401.4139534883721,348.7272727272727L402.49302325581397,348.7272727272727L403.5720930232558,348.7272727272727L404.6511627906977,348.7272727272727L405.7302325581395,348.7272727272727L406.80930232558137,348.7272727272727L407.8883720930233,348.7272727272727L408.9674418604651,348.7272727272727L410.0465116279069,348.7272727272727L411.12558139534883,348.7272727272727L412.2046511627907,348.7272727272727L413.28372093023256,348.7272727272727L414.3627906976744,348.7272727272727L415.44186046511624,348.7272727272727L416.52093023255816,348.7272727272727L417.59999999999997,348.7272727272727L418.6790697674419,348.7272727272727L419.75813953488375,348.7272727272727L420.83720930232556,348.7272727272727L421.9162790697675,348.7272727272727L422.9953488372093,348.7272727272727L424.0744186046511,348.7272727272727L425.153488372093,348.7272727272727L426.2325581395349,348.7272727272727L427.3116279069767,348.7272727272727L428.3906976744186,348.7272727272727L429.4697674418604,348.7272727272727L430.54883720930235,348.7272727272727L431.62790697674416,348.7272727272727L432.706976744186,348.7272727272727L433.78604651162794,348.7272727272727L434.86511627906975,348.7272727272727L435.9441860465117,348.7272727272727L437.0232558139535,348.7272727272727L438.10232558139535,325.81818181818176L439.1813953488372,325.81818181818176L440.2604651162791,325.81818181818176L441.3395348837209,325.81818181818176L442.4186046511628,325.81818181818176L443.4976744186046,325.81818181818176L444.5767441860465,325.81818181818176L445.65581395348835,325.81818181818176L446.7348837209302,325.81818181818176L447.81395348837214,325.81818181818176L448.89302325581394,325.81818181818176L449.97209302325575,325.81818181818176L451.0511627906977,325.81818181818176L452.13023255813954,325.81818181818176L453.2093023255814,325.81818181818176L454.28837209302327,325.81818181818176L455.3674418604651,325.81818181818176L456.446511627907,325.81818181818176L457.5255813953488,325.81818181818176L458.6046511627907,325.81818181818176L459.6837209302326,325.81818181818176L460.7627906976744,325.81818181818176L461.8418604651162,325.81818181818176L462.92093023255813,325.81818181818176L464,325.81818181818176L465.07906976744187,325.81818181818176L466.15813953488373,325.81818181818176L467.23720930232554,325.81818181818176L468.31627906976746,325.81818181818176L469.39534883720927,325.81818181818176L470.4744186046512,325.81818181818176L471.553488372093,325.81818181818176L472.63255813953486,325.81818181818176L473.7116279069768,325.81818181818176L474.7906976744186,325.81818181818176L475.8697674418604,325.81818181818176L476.9488372093023,325.81818181818176L478.0279069767442,325.81818181818176L479.106976744186,325.81818181818176L480.1860465116279,325.81818181818176L481.26511627906973,325.81818181818176L482.34418604651165,325.81818181818176L483.42325581395346,348.7272727272727L484.5023255813953,348.7272727272727L485.58139534883725,348.7272727272727L486.66046511627906,348.7272727272727L487.739534883721,348.7272727272727L488.8186046511628,348.7272727272727L489.89767441860465,348.7272727272727L490.9767441860465,348.7272727272727L492.0558139534884,348.7272727272727L493.1348837209302,348.7272727272727L494.2139534883721,348.7272727272727L495.2930232558139,348.7272727272727L496.3720930232558,348.7272727272727L497.45116279069765,348.7272727272727L498.5302325581395,348.7272727272727L499.60930232558144,348.7272727272727L500.68837209302325,348.7272727272727L501.76744186046506,348.7272727272727L502.846511627907,348.7272727272727L503.92558139534884,348.7272727272727L505.0046511627907,348.7272727272727L506.0837209302326,348.7272727272727L507.1627906976744,348.7272727272727L508.2418604651163,348.7272727272727L509.3209302325581,348.7272727272727L510.4,348.7272727272727L511.4790697674419,348.7272727272727L512.5581395348837,348.7272727272727L513.6372093023256,348.7272727272727L514.7162790697674,348.7272727272727L515.7953488372093,348.7272727272727L516.8744186046512,348.7272727272727L517.953488372093,348.7272727272727L519.0325581395349,348.7272727272727L520.1116279069768,348.7272727272727L521.1906976744186,348.7272727272727L522.2697674418605,348.7272727272727L523.3488372093024,348.7272727272727L524.4279069767441,325.81818181818176L525.5069767441861,325.81818181818176L526.586046511628,325.81818181818176L527.6651162790697,325.81818181818176L528.7441860465117,325.81818181818176L529.8232558139534,325.81818181818176L530.9023255813953,325.81818181818176L531.9813953488372,325.81818181818176L533.060465116279,325.81818181818176L534.1395348837209,325.81818181818176L535.2186046511628,325.81818181818176L536.2976744186046,325.81818181818176L537.3767441860465,325.81818181818176L538.4558139534884,325.81818181818176L539.5348837209302,325.81818181818176L540.6139534883721,325.81818181818176L541.693023255814,325.81818181818176L542.7720930232558,325.81818181818176L543.8511627906977,325.81818181818176L544.9302325581396,325.81818181818176L546.0093023255814,325.81818181818176L547.0883720930233,325.81818181818176L548.167441860465,325.81818181818176L549.246511627907,325.81818181818176L550.3255813953488,325.81818181818176L551.4046511627907,325.81818181818176L552.4837209302325,325.81818181818176L553.5627906976744,325.81818181818176L554.6418604651163,325.81818181818176L555.7209302325581,325.81818181818176L556.8000000000001,325.81818181818176L557.8790697674418,325.81818181818176L558.9581395348837,325.81818181818176L560.0372093023256,325.81818181818176L561.1162790697674,325.81818181818176L562.1953488372093,325.81818181818176L563.2744186046511,325.81818181818176L564.353488372093,325.81818181818176L565.4325581395349,325.81818181818176L566.5116279069767,325.81818181818176L567.5906976744186,325.81818181818176L568.6697674418605,325.81818181818176L569.7488372093023,325.81818181818176L570.8279069767442,325.81818181818176L571.9069767441861,325.81818181818176L572.9860465116279,325.81818181818176L574.0651162790698,325.81818181818176L575.1441860465117,325.81818181818176L576.2232558139534,325.81818181818176L577.3023255813954,325.81818181818176L578.3813953488371,325.81818181818176L579.460465116279,325.81818181818176L580.539534883721,325.81818181818176L581.6186046511627,325.81818181818176L582.6976744186046,325.81818181818176L583.7767441860465,325.81818181818176L584.8558139534883,325.81818181818176L585.9348837209302,325.81818181818176L587.0139534883721,325.81818181818176L588.0930232558139,325.81818181818176L589.1720930232558,325.81818181818176L590.2511627906977,325.81818181818176L591.3302325581395,325.81818181818176L592.4093023255814,325.81818181818176L593.4883720930233,325.81818181818176L594.5674418604651,325.81818181818176L595.646511627907,325.81818181818176L596.7255813953489,325.81818181818176L597.8046511627907,325.81818181818176L598.8837209302326,325.81818181818176L599.9627906976743,325.81818181818176L601.0418604651163,325.81818181818176L602.1209302325581,325.81818181818176L603.2,325.81818181818176L604.2790697674418,325.81818181818176L605.3581395348837,325.81818181818176L606.4372093023256,325.81818181818176L607.5162790697674,325.81818181818176L608.5953488372094,325.81818181818176L609.6744186046511,325.81818181818176L610.753488372093,325.81818181818176L611.8325581395349,325.81818181818176L612.9116279069767,325.81818181818176L613.9906976744186,325.81818181818176L615.0697674418604,325.81818181818176L616.1488372093023,325.81818181818176L617.2279069767442,325.81818181818176L618.306976744186,325.81818181818176L619.3860465116279,325.81818181818176L620.4651162790698,325.81818181818176L621.5441860465116,325.81818181818176L622.6232558139535,325.81818181818176L623.7023255813954,325.81818181818176L624.7813953488372,325.81818181818176L625.8604651162791,325.81818181818176L626.939534883721,325.81818181818176L628.0186046511627,325.81818181818176L629.0976744186047,325.81818181818176L630.1767441860464,325.81818181818176L631.2558139534883,325.81818181818176L632.3348837209303,325.81818181818176L633.413953488372,325.81818181818176L634.4930232558139,302.9090909090909L635.5720930232558,302.9090909090909L636.6511627906976,302.9090909090909L637.7302325581395,302.9090909090909L638.8093023255814,302.9090909090909L639.8883720930232,302.9090909090909L640.9674418604651,302.9090909090909L642.046511627907,302.9090909090909L643.1255813953488,302.9090909090909L644.2046511627907,302.9090909090909L645.2837209302326,302.9090909090909L646.3627906976744,302.9090909090909L647.4418604651163,302.9090909090909L648.5209302325582,302.9090909090909L649.6,302.9090909090909L650.6790697674419,302.9090909090909L651.7581395348836,302.9090909090909L652.8372093023256,302.9090909090909L653.9162790697674,302.9090909090909L654.9953488372093,302.9090909090909L656.0744186046511,302.9090909090909L657.153488372093,302.9090909090909L658.232558139535,302.9090909090909L659.3116279069767,302.9090909090909L660.3906976744187,302.9090909090909L661.4697674418604,302.9090909090909L662.5488372093023,302.9090909090909L663.6279069767442,302.9090909090909L664.706976744186,302.9090909090909L665.7860465116279,302.9090909090909L666.8651162790698,280L667.9441860465116,280L669.0232558139535,280L670.1023255813953,280L671.1813953488372,280L672.2604651162791,280L673.339534883721,280L674.4186046511627,280L675.4976744186047,280L676.5767441860465,280L677.6558139534884,280L678.7348837209303,280L679.813953488372,280L680.893023255814,280L681.9720930232558,280L683.0511627906976,280L684.1302325581396,280L685.2093023255813,280L686.2883720930232,280L687.3674418604651,280L688.446511627907,280L689.5255813953488,280L690.6046511627907,280L691.6837209302325,280L692.7627906976744,280L693.8418604651163,280L694.9209302325581,280L696,280"></path></g><line class="simple-line avg-line velocity_smooth" x1="0" y1="31.67615917231712" x2="696" y2="31.67615917231712"></line><line class="simple-line avg-line watts" x1="0" y1="109.33243525169135" x2="696" y2="109.33243525169135"></line><line class="simple-line avg-line heartrate" x1="0" y1="157.02492573937792" x2="696" y2="157.02492573937792"></line><line class="simple-line avg-line cadence" x1="0" y1="234.32728681624462" x2="696" y2="234.32728681624462"></line><line class="simple-line avg-line temp" x1="0" y1="322.6760593145706" x2="696" y2="322.6760593145706"></line><line class="bottom-line partial" x1="-30" y1="70" x2="696" y2="70"></line><line class="bottom-line partial" x1="-30" y1="140" x2="696" y2="140"></line><line class="bottom-line partial" x1="-30" y1="210" x2="696" y2="210"></line><line class="bottom-line partial" x1="-30" y1="280" x2="696" y2="280"></line><line class="bottom-line partial" x1="-30" y1="350" x2="696" y2="350"></line><g id="brush" style="pointer-events: none;"><rect class="background" x="0" width="696" height="420" style="visibility: hidden; cursor: crosshair;"></rect><rect class="extent" x="45.82354736328125" width="19" height="420" style="cursor: move;"></rect><g class="resize e" transform="translate(64.82354736328125,0)" style="cursor: ew-resize;"><rect x="-3" width="6" height="420" style="visibility: hidden;"></rect></g><g class="resize w" transform="translate(45.82354736328125,0)" style="cursor: ew-resize;"><rect x="-3" width="6" height="420" style="visibility: hidden;"></rect></g></g><g id="crossBar" display="none"><line x1="114.32354736328125" x2="114.32354736328125" y1="350" y2="0" class="crossbar" display=""></line><text x="124.82354736328125" y="15" class="crossbar-text" text-anchor="left" display="" pointer-events="none"></text></g></g><g class="labelGroup" transform="translate(0, 30)"><g class="label-box" transform="translate(0, 1)"><rect class="static-label-box" x="0" y="0" width="120" height="69"></rect><text class="static-label-box label" x="60" y="0" dy="2em">Speed</text><text class="static-label-box detail max-js" x="60" y="0" dy="4em">Max 47.0</text><text class="static-label-box detail avg-js" x="60" y="0" dy="5.5em">Avg 27.5</text><text class="static-label-box top" x="125" y="15">47.0</text><text class="static-label-box bottom" x="125" y="65">0.0</text></g><g class="label-box" transform="translate(0, 71)"><rect class="static-label-box" x="0" y="0" width="120" height="69"></rect><text class="static-label-box label" x="60" y="0" dy="2em">Power</text><text class="static-label-box detail max-js" x="60" y="0" dy="4em">Max 649</text><text class="static-label-box detail avg-js" x="60" y="0" dy="5.5em">Avg 285</text><text class="static-label-box top" x="125" y="15">649</text><text class="static-label-box bottom" x="125" y="65">0</text></g><g class="label-box" transform="translate(0, 141)"><rect class="static-label-box" x="0" y="0" width="120" height="69"></rect><text class="static-label-box label" x="60" y="0" dy="2em">Heart Rate</text><text class="static-label-box detail max-js" x="60" y="0" dy="4em">Max 203</text><text class="static-label-box detail avg-js" x="60" y="0" dy="5.5em">Avg 188</text><text class="static-label-box top" x="125" y="15">203</text><text class="static-label-box bottom" x="125" y="65">129</text></g><g class="label-box" transform="translate(0, 211)"><rect class="static-label-box" x="0" y="0" width="120" height="69"></rect><text class="static-label-box label" x="60" y="0" dy="2em">Cadence</text><text class="static-label-box detail max-js" x="60" y="0" dy="4em">Max 139</text><text class="static-label-box detail avg-js" x="60" y="0" dy="5.5em">Avg 102</text><text class="static-label-box top" x="125" y="15">139</text><text class="static-label-box bottom" x="125" y="65">35</text></g><g class="label-box" transform="translate(0, 281)"><rect class="static-label-box" x="0" y="0" width="120" height="69"></rect><text class="static-label-box label" x="60" y="0" dy="2em">Temperature</text><text class="static-label-box detail max-js" x="60" y="0" dy="4em">Max 68</text><text class="static-label-box detail avg-js" x="60" y="0" dy="5.5em">Avg 65</text><text class="static-label-box top" x="125" y="15">68</text><text class="static-label-box bottom" x="125" y="65">63</text></g></g><g class="listenerGroup" transform="translate(846, 30)"><g class="listener-box" transform="translate(0, 1)"><rect class="static-info-box" x="0" y="0" width="120" height="69"></rect><text class="static-info-box top" x="60" y="19"></text><text class="static-info-box bottom" x="60" y="55">mi/h</text><text class="static-info-box value" x="60" y="35" dy="0.3em">--</text></g><g class="listener-box" transform="translate(0, 71)"><rect class="static-info-box" x="0" y="0" width="120" height="69"></rect><text class="static-info-box top" x="60" y="19"></text><text class="static-info-box bottom" x="60" y="55">W</text><text class="static-info-box value" x="60" y="35" dy="0.3em">--</text></g><g class="listener-box" transform="translate(0, 141)"><rect class="static-info-box" x="0" y="0" width="120" height="69"></rect><text class="static-info-box top" x="60" y="19"></text><text class="static-info-box bottom" x="60" y="55">bpm</text><text class="static-info-box value" x="60" y="35" dy="0.3em">--</text></g><g class="listener-box" transform="translate(0, 211)"><rect class="static-info-box" x="0" y="0" width="120" height="69"></rect><text class="static-info-box top" x="60" y="19"></text><text class="static-info-box bottom" x="60" y="55">rpm</text><text class="static-info-box value" x="60" y="35" dy="0.3em">--</text></g><g class="listener-box" transform="translate(0, 281)"><rect class="static-info-box" x="0" y="0" width="120" height="69"></rect><text class="static-info-box top" x="60" y="19"></text><text class="static-info-box bottom" x="60" y="55">℉</text><text class="static-info-box value" x="60" y="35" dy="0.3em">--</text></g></g><defs><clipPath id="clip534"><rect id="clip534" x="0" y="0" height="420" width="696"></rect></clipPath></defs><defs><clipPath id="clip132"><rect id="clip132" x="0" y="0" height="420" width="696"></rect></clipPath></defs><defs><clipPath id="clip373"><rect id="clip373" x="0" y="0" height="420" width="696"></rect></clipPath></defs><defs><clipPath id="clip74"><rect id="clip74" x="0" y="0" height="420" width="696"></rect></clipPath></defs><defs><clipPath id="clip79"><rect id="clip79" x="0" y="0" height="420" width="696"></rect></clipPath></defs><g class="xaxis-container"><rect x="0" y="0" width="966" height="30" class="background"></rect><g class="chart-controls" transform="translate(10,3)"><g class="button" data-type="distance" transform="translate(0,0)"><title>Distance</title><rect height="24" width="35"></rect><path d="M0 24 L0 0 L35 0"></path><image height="16" width="35" x="0" y="4" xlink:href="/assets/svg/icon-distance.svg"></image></g><g class="button selected" data-type="time" transform="translate(36,0)"><title>Time</title><rect height="24" width="35"></rect><path d="M0 24 L0 0 L35 0"></path><image height="16" width="35" x="0" y="4" xlink:href="/assets/svg/icon-time.svg"></image></g></g><g class="axis xaxis" id="xaxis" transform="translate(150, 30)"><g class="tick" transform="translate(0,0)" style="opacity: 1;"><line y2="-6" x2="0"></line><text y="-9" x="0" dy="0em" style="text-anchor: middle;">0s</text></g><g class="tick" transform="translate(53.95348837209302,0)" style="opacity: 1;"><line y2="-6" x2="0"></line><text y="-9" x="0" dy="0em" style="text-anchor: middle;">3:20</text></g><g class="tick" transform="translate(107.90697674418604,0)" style="opacity: 1;"><line y2="-6" x2="0"></line><text y="-9" x="0" dy="0em" style="text-anchor: middle;">6:40</text></g><g class="tick" transform="translate(161.86046511627907,0)" style="opacity: 1;"><line y2="-6" x2="0"></line><text y="-9" x="0" dy="0em" style="text-anchor: middle;">10:00</text></g><g class="tick" transform="translate(215.81395348837208,0)" style="opacity: 1;"><line y2="-6" x2="0"></line><text y="-9" x="0" dy="0em" style="text-anchor: middle;">13:20</text></g><g class="tick" transform="translate(269.7674418604651,0)" style="opacity: 1;"><line y2="-6" x2="0"></line><text y="-9" x="0" dy="0em" style="text-anchor: middle;">16:40</text></g><g class="tick" transform="translate(323.72093023255815,0)" style="opacity: 1;"><line y2="-6" x2="0"></line><text y="-9" x="0" dy="0em" style="text-anchor: middle;">20:00</text></g><g class="tick" transform="translate(377.6744186046511,0)" style="opacity: 1;"><line y2="-6" x2="0"></line><text y="-9" x="0" dy="0em" style="text-anchor: middle;">23:20</text></g><g class="tick" transform="translate(431.62790697674416,0)" style="opacity: 1;"><line y2="-6" x2="0"></line><text y="-9" x="0" dy="0em" style="text-anchor: middle;">26:40</text></g><g class="tick" transform="translate(485.58139534883725,0)" style="opacity: 1;"><line y2="-6" x2="0"></line><text y="-9" x="0" dy="0em" style="text-anchor: middle;">30:00</text></g><g class="tick" transform="translate(539.5348837209302,0)" style="opacity: 1;"><line y2="-6" x2="0"></line><text y="-9" x="0" dy="0em" style="text-anchor: middle;">33:20</text></g><g class="tick" transform="translate(593.4883720930233,0)" style="opacity: 1;"><line y2="-6" x2="0"></line><text y="-9" x="0" dy="0em" style="text-anchor: middle;">36:40</text></g><g class="tick" transform="translate(647.4418604651163,0)" style="opacity: 1;"><line y2="-6" x2="0"></line><text y="-9" x="0" dy="0em" style="text-anchor: middle;">40:00</text></g><path class="domain" d="M0,-6V0H696V-6"></path></g></g></svg></div></div>'
soup = BeautifulSoup(html3, 'html.parser')
extract_mean_max_metrics(soup)

{'temp_min': 63.09090909090909,
 'temp_max': 68.0,
 'speed_average': 25.820781664042347,
 'speed_maximum': 39.373434129556806,
 'power_average': 282.65700801531284,
 'power_maximum': 630.2788461538462,
 'hr_average': 185.05944358043897,
 'hr_maximum': 197.2444444444444,
 'cadence_average': 102.84408313304112,
 'cadence_maximum': 132.38181818181818,
 'speed_5_seconds': 37.901530049947205,
 'speed_10_seconds': 36.83439959223024,
 'speed_12_seconds': 36.819505324758005,
 'speed_20_seconds': 36.8107439909508,
 'speed_30_seconds': 35.77077366803615,
 'speed_1_minute': 34.87448921955959,
 'speed_2_minutes': 34.479353064854884,
 'speed_5_minutes': 33.01025261206358,
 'speed_6_minutes': 32.4724235340856,
 'speed_10_minutes': 32.543010680125604,
 'speed_12_minutes': 30.865448891615,
 'speed_20_minutes': 29.38691540275795,
 'speed_30_minutes': 23.384817655905476,
 'speed_1_hour': None,
 'speed_90_minutes': None,
 'speed_3_hours': None,
 'power_5_seconds': 607.1145384615386,
 'power_10_seconds': 

### Extract points from elevation graph and calculate the elevation stats (min, max, ...)

In [10]:
def extract_graph_elevation_distance(soup):
     # Graph data (height...)
    all_points = extract_points_from_graph(soup)
    
    def calculate_distance_for_height(points, min_dis, max_dis):
        in_range = False
        relevent_points = []
        for x, y in points:

            if x >= min_dis and x < max_dis:
                if not in_range:
                    relevent_points.append([])
                    in_range = True
                relevent_points[-1].append((x,y))
            else:
                in_range = False
        total_distance = 0
        for section in relevent_points:
            first_point = section[0]
            end_point = section[-1]
            total_distance += end_point[1] - first_point[1]

        return total_distance
    
    def calculate_elevation_gain_loss(points):
        
        elevation_gain = 0
        elevation_loss = 0
        max_ = float('-inf')
        min_ = float('inf')
        elevating_now = 0
        up = True
        for i in range(1, len(points)):
            prev = points[i-1][0] * 0.3048 # ft to meter
            curr = points[i][0] * 0.3048 # ft to meter
            
            
            
                
            if prev > curr:
                if up:
                    min_ = min(elevating_now, min_)
                    elevating_now = 0
                up = False
                elevation_loss += (prev - curr)
                elevating_now += (prev - curr)
            elif prev < curr:
                if not up:
                    max_ = max(elevating_now, max_)
                    elevating_now = 0
                up = True
                elevation_gain += (curr - prev)
                elevating_now += (curr - prev)
            
            
        return elevation_gain, elevation_loss, min_, max_
    
    def calculate_elevation_mean_max_min(points):
        Y = []
        for point in points:
            Y.append(point[0])
        return np.min(Y), np.max(Y), np.mean(Y)
            
                    
    # Calculate the props
    # workouts table
    dic = {}
    dic['_1000_to_1500_m'] = calculate_distance_for_height(all_points, 1000, 1500)
    dic['_1500_to_2000_m'] = calculate_distance_for_height(all_points, 1500, 2000)
    dic['_2000_tp_2500_m'] = calculate_distance_for_height(all_points, 2000, 2500)
    dic['_2500_to_3000_m'] = calculate_distance_for_height(all_points, 2500, 3000)
    dic['_3000_to_3500_m'] = calculate_distance_for_height(all_points, 3000, 3500)
    
    elevation_gain, elevation_loss, min_, max_ = calculate_elevation_gain_loss(all_points)
    dic['elevation_gain'] = elevation_gain
    dic['elevation_loss'] = elevation_loss
#     dic['elevation_minimum'] = min_
#     dic['elevation_maximum'] = max_
    
    min_, max_, mean_ = calculate_elevation_mean_max_min(all_points)
    dic['elevation_minimum'] = min_
    dic['elevation_maximum'] = max_
    dic['elevation_average'] = mean_
    
    return dic

### Extract data from the home activity page

In [12]:
def extract_activity_home(url):
    data = {}
    browser.get(url)
    t.sleep(0.2)
    print(browser.current_url)
    
    activity_soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    data['type'] = activity_soup.find('span', {'class': 'title'}).text.split('\n')[-2]
    data['workout_strava_id'] = re.findall('.*/([0-9]+)$',url)[0]
    
    strong_data = activity_soup.find_all("strong")
    strong_data = BeautifulSoup(str(strong_data), "html.parser").get_text()
    strong_data.split('\\n')
    strong_data = strong_data.strip('][').split(', ')
    strong_data = list(map(lambda x:x.strip("\n"), strong_data))
    
    strong_data_labels = activity_soup.find_all('div', {'class': 'label'})
    strong_data_labels = BeautifulSoup(str(strong_data_labels), "html.parser").get_text()
    strong_data_labels = strong_data_labels.split('\\n')
    strong_data_labels = strong_data_labels[0].strip('][').split(', ')
    strong_data_labels = list(map(lambda x:x.strip("\n").lower(), strong_data_labels))
    
    
    
    strong_dict = dict(zip(strong_data_labels, strong_data))
    
    if 'distance' in strong_dict.keys():
        data['distance'] = float(strong_dict['distance'][:-2])*1609.34
    else:
        data['distance'] = None
        
    if 'moving time' in strong_dict.keys():
        time = strong_dict['moving time'].split(':')
        total_time = 0
        for i in range(len(time)):
            total_time += int(time[i])/(60**i)
        data['total_time'] = total_time
    else:
        data['total_time'] = None
    
    
    if 'relative effort' in strong_dict.keys():
        data['relative_effort'] = float(strong_dict['relative effort'])
    else:
        data['relative_effort'] = None
        
    if 'total work' in strong_dict.keys():
        data['energy'] = float(strong_dict['total work'][:-2])
    else:
        data['energy'] = None
        
    if 'training load' in strong_dict.keys():
        data['training_load'] = float(strong_dict['training load'])
    else:
        data['training_load'] = None
    
    if 'intensity' in strong_dict.keys():
        data['intensity'] = float(strong_dict['intensity'][:-1])
    else:
        data['intensity'] = None
        
    if 'relative effort' in strong_dict.keys():
        data['relative_effort'] = float(strong_dict['relative effort'])
    else:
        data['relative_effort'] = None
        
    date = activity_soup.find('time').text.replace('\n', '')
    date = datetime.strptime(date, '%A, %B %d, %Y')
    data['workout_datetime'] = date
    data['workout_week'] = date.isocalendar()[1]
    data['workout_month'] = date.month
    data['temp_avg'] = (float(re.sub('\D', '', activity_soup.find('div', {'class': 'weather-value'}).text)) - 32 ) * (5/9) 
    data['temp_min'] = None
    data['temp_max'] = None
    
    
    tr = activity_soup.find_all("tr")
    tr_datas = BeautifulSoup(str(tr), "html.parser").get_text()
    tr_datas = tr_datas.strip('][').split(', ')[1:]
    tr_datas = list(map(lambda x:x.split('\n'), tr_datas))
    tr_datas = list(map(lambda dt :[x for x in dt if x], tr_datas))
    
    tr_dict = {}
    
    for dt in tr_datas:
        
        name = dt[0]
        
        if name == 'Speed':
            tr_dict['speed_average'] = float(dt[1][:-4])
            tr_dict['speed_maximum'] = float(dt[2][:-4])
            
        elif name == 'Heart Rate':
            tr_dict['hr_average'] = float(dt[1][:-3])
            tr_dict['hr_maximum'] = float(dt[2][:-3])

        elif name == 'Calories':
            tr_dict['calories'] = float(dt[1].replace(',', ''))
             
        elif name == 'Cadence':
            tr_dict['cadence_average'] = float(dt[1])
            tr_dict['cadence_maximum'] = float(dt[2])

        elif name == 'Power':
            tr_dict['power_avg'] = float(dt[1][:-1])
            tr_dict['power_max'] = float(dt[2][:-1])
     
        elif name == 'Temperature':
            pass
               
        elif name == 'Elapsed Time':
            pass
        else:
            break
            
    if 'calories' in tr_dict.keys():  
        data['calories'] = tr_dict['calories']
    else:
        data['calories'] = None
            
    data['workout_title'] = activity_soup.find('h1').text
    data['athelete_name'] = activity_soup.find('a', {'class':'minimal'}).text
    data['normalized_power'] = None
    data['IF'] = None
    data['tss_actual'] = None
    data['tss_calculation_method'] = None
    data['hidden'] = None
    data['locked'] = None
    return data
    

### Extract data from the analysis activity page

In [13]:
def extract_activity_analysis(url):
    
    data = {}
    browser.get(url + '/analysis')
    print(browser.current_url)
    t.sleep(0.2)
    analysis_soup = BeautifulSoup(browser.page_source, 'html.parser')
    chart = analysis_soup.find_all('g', {'class': "label-box"})
    svg = analysis_soup.find_all('defs')
    while len(chart) == 0 or len(svg) == 0: # Wait for load
        analysis_soup = BeautifulSoup(browser.page_source, 'html.parser')
        chart = analysis_soup.find_all('g', {'class': "label-box"})
        svg = analysis_soup.find_all('defs')
        t.sleep(0.1)
        
    
    # Raw data (speed, est power, heart rate)
    for label_box in chart:
        text_data = []
        for text in label_box.find_all('text'):
            text_data.append(text.get_text())
            
            
        name = text_data[0].lower().replace(' ', '_')
        max_attribute = text_data[1].split(' ')[1]
        avg_attribute = text_data[2].split(' ')[1]
        
        
        if name == 'heart_rate':
            name = 'hr'
    
        elif name == 'temperature':
            name = 'temp'

        
        data[f'{name}_maximum'] = max_attribute
        data[f'{name}_average'] = avg_attribute
    
    
    e_g_e_d = extract_graph_elevation_distance(analysis_soup)
    data.update(e_g_e_d)
      
    button = browser.find_element_by_css_selector("g[data-type='time']")

    button.click()
    t.sleep(0.2)

    timeout = t.time() + 5
    loading = True
    while(loading):
        t.sleep(0.5)

        analysis_soup = BeautifulSoup(browser.page_source, 'html.parser')
        axis = analysis_soup.find('g', {'class':'axis xaxis'})
        tick = axis.find_all('g', {'class': 'tick'})[1]
        time = list(reversed(tick.find('text').text.replace('s', '').split(':')))[0]
        
        try:
            int(time)
            loading = False
        except:
            if t.time() > timeout:
                break
        
            
    if not loading:    
        analysis_soup = BeautifulSoup(browser.page_source, 'html.parser')
        e_m_m_m = extract_mean_max_metrics(analysis_soup)
        data.update(e_m_m_m)
    return data

### Extract data from the heart-rate and the zones-distribution activity pages

In [33]:
def extract_activity_zones_distribution(url):
    url_extentions = ['/zone-distribution', '/heartrate']
    prefixes = ['power', 'hr']
    data = {}
    j = 0
    for extension in url_extentions:
        browser.get(url + extension)
        print(browser.current_url)
        t.sleep(0.2)
        zone_soup = BeautifulSoup(browser.page_source, 'html.parser')
        zones = zone_soup.find_all('tr')
        while len(zones) is None:
            zone_soup = BeautifulSoup(browser.page_source, 'html.parser')
            zones = zone_soup.find_all('tr')
            t.sleep(0.1)

        zone_dist_dict = {}

        for zone in zones:
            z = [x for x in zone.get_text().strip().split('\n') if x]

            if not z[0].startswith("Z"):
                continue

            z_name = z[0]
            z_min = z[2].split(' ')
            z_min = [re.sub('[^0-9]', '', x) for x in z_min]
            z_min = [x for x in z_min if x]
            z_min = float(z_min[0])
            

            z_time = to_seconds(z[-2])
            zone_dist_dict[f'{prefixes[j]}_{z_name}'.lower()] = z_time
            zone_dist_dict[f'{prefixes[j]}_{z_name}_min'.lower()] = z_min

    
        for i in range(1,8):
            if f'{prefixes[j]}_z{i}' in zone_dist_dict:
                data[f'{prefixes[j]}_zone_{i}'] = zone_dist_dict[f'{prefixes[j]}_z{i}']
                data[f'{prefixes[j]}_zone_{i}_min'] = zone_dist_dict[f'{prefixes[j]}_z{i}_min']
            else:
                data[f'{prefixes[j]}_zone_{i}'] = None
                data[f'{prefixes[j]}_zone_{i}_min'] = None
        j += 1

    return data

### Return the data collected from the activity in table chunks

In [40]:
def divide_to_tables(data):
    # columns!
    workout_columns = ['workout_id' ,'workout_tp_id','workout_strava_id', 'cyclist_id', 'type' ,'tags', 'workout_week', 'workout_month', 'workout_datetime', 'total_time', 'workout_title', 'cyclist_mass', 'elevation_gain', 'elevation_loss', 'elevation_average', 'elevation_maximum', 'elevation_minimum', 'temp_avg', 'temp_min', 'temp_max','_1000_to_1500_m', '_1500_to_2000_m', '_2000_tp_2500_m', '_2500_to_3000_m', '_3000_to_3500_m', 'relative_effort', 'training_load', 'intensity', 'distance', 'energy', 'calories', 'if', 'tss_actual', 'tss_calculation_method', 'hidden','locked']
    workout_hrs_columns = ['workout_id', 'hr_maximum', 'hr_average','hr_5_seconds', 'hr_10_seconds', 'hr_12_seconds', 'hr_20_seconds', 'hr_30_seconds', 'hr_1_minute', 'hr_2_minutes', 'hr_5_minutes', 'hr_6_minutes', 'hr_10_minutes', 'hr_12_minutes', 'hr_20_minutes', 'hr_30_minutes', 'hr_1_hour', 'hr_90_minutes', 'hr_3_hours', 'hr_zone_1', 'hr_zone_2', 'hr_zone_3', 'hr_zone_4', 'hr_zone_5', 'hr_zone_1_min', 'hr_zone_2_min', 'hr_zone_3_min', 'hr_zone_4_min' ,'hr_zone_5_min']
    workout_cadences_columns = ['workout_id', 'cadence_5_seconds', 'cadence_10_seconds', 'cadence_12_seconds', 'cadence_20_seconds', 'cadence_30_seconds', 'cadence_1_minute', 'cadence_2_minutes', 'cadence_5_minutes', 'cadence_6_minutes', 'cadence_10_minutes', 'cadence_12_minutes', 'cadence_20_minutes', 'cadence_30_minutes', 'cadence_1_hour', 'cadence_90_minutes', 'cadence_3_hours', 'cadence_maximum', 'cadence_average']
    workout_powers_columns = ['workout_id', 'power_maximum', 'power_average', 'normalized_power', 'power_5_seconds', 'power_10_seconds', 'power_12_seconds', 'power_20_seconds', 'power_30_seconds', 'power_1_minute', 'power_2_minutes', 'power_5_minutes', 'power_6_minutes', 'power_10_minutes', 'power_12_minutes', 'power_20_minutes', 'power_30_minutes', 'power_1_hour', 'power_90_minutes', 'power_3_hours', 'power_zone_1', 'power_zone_2', 'power_zone_3', 'power_zone_4', 'power_zone_5', 'power_zone_6', 'power_zone_7', 'power_zone_1_min', 'power_zone_2_min', 'power_zone_3_min', 'power_zone_4_min', 'power_zone_5_min', 'power_zone_6_min', 'power_zone_7_min']
    workout_speeds_columns = ['workout_id', 'speed_maximum', 'speed_average', 'speed_5_seconds', 'speed_10_seconds', 'speed_12_seconds', 'speed_20_seconds', 'speed_30_seconds', 'speed_1_minute', 'speed_2_minutes', 'speed_5_minutes', 'speed_6_minutes', 'speed_10_minutes', 'speed_12_minutes', 'speed_20_minutes', 'speed_30_minutes', 'speed_1_hour', 'speed_90_minutes', 'speed_3_hours', 'speed_zone_1', 'speed_zone_2', 'speed_zone_3', 'speed_zone_4', 'speed_zone_5', 'speed_zone_6', 'speed_zone_7', 'speed_zone_1_min', 'speed_zone_2_min', 'speed_zone_3_min', 'speed_zone_4_min', 'speed_zone_5_min', 'speed_zone_6_min', 'speed_zone_7_min']
    
    
    #workout
    workout_row = {}
    workout_hrs_row = {}
    workout_cadences_row = {}
    workout_powers_row = {}
    workout_speeds_row = {}
    key_not_found = []
    for key, value in data.items():
        key = key.lower()
        found = False
        if key in workout_columns:
            workout_row[key] = value
            found = True
        if key in workout_hrs_columns:
            workout_hrs_row[key] = value
            found = True
        if key in workout_cadences_columns:
            workout_cadences_row[key] = value
            found = True
        if key in workout_powers_columns:
            workout_powers_row[key] = value
            found = True
        if key in workout_speeds_columns:
            workout_speeds_row[key] = value
            found = True
        if not found:
            key_not_found.append(key)
    
    return workout_row, workout_hrs_row, workout_cadences_row, workout_powers_row, workout_speeds_row, key_not_found
            


### Extract the activity data from the url

In [16]:
def extract_activity_data(url):
    
    data = {}
    home = extract_activity_home(url)
    analysis = extract_activity_analysis(url)
    zones_distribution = extract_activity_zones_distribution(url)
    
    data.update(home)
    data.update(analysis)
    data.update(zones_distribution)
            
    
    return divide_to_tables(data)
    

## Extract all activities data from riders

In [17]:
# prepare browser
browser = chrome_driver()
browser.current_url

'https://www.strava.com/onboarding'

In [ ]:
riders_pickle =  open('data/riders.pickle', 'rb')
riders = pk.load(riders_pickle)

In [ ]:
for rider in riders:
    for link in rider.activity_links:
        workout_row, workout_hrs_row, workout_cadences_row, workout_powers_row, workout_speeds_row, key_not_found = extract_activity_data(link)
        rider.workout.append(workout_row)
        rider.workout_hrs.append(workout_hrs_row)
        rider.workout_cadences.append(workout_cadences_row)
        rider.workout_powers.append(workout_powers_row)
        rider.workout_speeds.append(workout_speeds_row)

In [ ]:
with open('data/riders.pickle', 'wb') as handle:
    pk.dump(riders, handle, protocol=pk.HIGHEST_PROTOCOL)

## Debugging section

In [54]:
url1 = 'https://www.strava.com/activities/4257277756'
url2 = 'https://www.strava.com/activities/4296963794'
url3 = 'https://www.strava.com/activities/5650998097'
workout_row, workout_hrs_row, workout_cadences_row, workout_powers_row, workout_speeds_row, key_not_found = extract_activity_data(url2)

https://www.strava.com/activities/4296963794
https://www.strava.com/activities/4296963794/analysis


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


https://www.strava.com/activities/4296963794/zone-distribution
https://www.strava.com/activities/4296963794/heartrate


In [70]:
#workout_cadences_row
workout_powers_row
# workout_hrs_row
# workout_speeds_row

{'normalized_power': None,
 'power_maximum': 630.2788461538462,
 'power_average': 282.65700801531284,
 'power_5_seconds': 299.98520309130123,
 'power_10_seconds': 311.93995378012545,
 'power_12_seconds': 316.1182448262398,
 'power_20_seconds': 327.69998738965955,
 'power_30_seconds': 338.2197138836773,
 'power_1_minute': 358.0197802197802,
 'power_2_minutes': 375.4571978021978,
 'power_5_minutes': 400.38307692307694,
 'power_6_minutes': 394.3923076923077,
 'power_10_minutes': 415.1103846153846,
 'power_12_minutes': 418.6882051282051,
 'power_20_minutes': 426.34307692307686,
 'power_30_minutes': 449.3076923076922,
 'power_1_hour': nan,
 'power_90_minutes': nan,
 'power_3_hours': nan,
 'power_zone_1': 14,
 'power_zone_1_min': 1.0,
 'power_zone_2': 258,
 'power_zone_2_min': 172.0,
 'power_zone_3': 964,
 'power_zone_3_min': 235.0,
 'power_zone_4': 995,
 'power_zone_4_min': 281.0,
 'power_zone_5': 273,
 'power_zone_5_min': 328.0,
 'power_zone_6': 59,
 'power_zone_6_min': 375.0,
 'power_zone

In [71]:
key_not_found#, workout_row

['athelete_name',
 'temp_maximum',
 'temp_average',
 'hr_zone_6',
 'hr_zone_6_min',
 'hr_zone_7',
 'hr_zone_7_min']